In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import  plotly
plotly.tools.set_credentials_file(username='RishiHazra', api_key='3WYShX1Rc0UlKTzCVggk')
from collections import Counter,OrderedDict
import plotly.offline as py
from plotly import tools
from plotly.graph_objs import *
py.init_notebook_mode(connected=True)
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from wordcloud import WordCloud, STOPWORDS
import matplotlib.image as mpimg
from PIL import Image

# About DonorsChoose.org

In [2]:
img=mpimg.imread('../input/masks2/donorschoose.jpg')
plt.figure(figsize=[14,8])
imgplot = plt.imshow(img)
plt.axis('Off')
plt.show()

## Data Science for Good : DonorsChoose
**DonorsChoose.org is a United States¨Cbased nonprofit organization that allows individuals to donate directly to public school classroom projects. Founded in 2000 by former public school teacher Charles Best, DonorsChoose.org was among the first civic crowdfunding platforms of its kind. The organization has been given Charity Navigator¡¯s highest rating every year since 2005. In 77% of public schools in the United States, at least one project has been requested on DonorsChoose.org. This notebook contains detailed analysis of the data provided to us that gives us a lot of insights to serve the donation process in a better way. Various kinds of state of the art recommender systems are also compared to find the best one.**

@source: wikipedia

# Table of Contents

* 1  [Teachers.csv](#1)    
    1.1 [A glimpse of Teacher.csv](#1.1)    
    1.2 [Teacher Prefix Pre-Processing](#1.2)     
    1.3 [Trend in Teacher First Project Posted Date](#1.3)        
    
   
* 2  [Schools.csv](#2)   
    2.1 [A glimpse of Schools.csv](#2.1)   
     2.1.1 [Null values in the dataset](#2.1.1)    
     2.1.2 [Replacing the Null values in the 'School Percentage Free Lunch' with the School District average](#2.1.2)      
    2.2 [School Type Distribution](#2.2)


* 3  [Donors.csv](#3)    
    3.1 [A glimpse of Donors.csv](#3.1)    
    3.2 [Using 'Donor Zip' to fill some 'nan' values in 'Donor State'](#3.2)    
    3.2.1 [  Replacing the 'Donor State' (other) for 'Donor Zip' (245) with 'Virginia'](#3.2.1)    



* 4  [Donations.csv](#4)   
    4.1 [A glimpse of Donations.csv](#4.1)    
    4.2 [Optional Donation information](#4.2)    
    4.3 [Comparing 'Total Donation Amount' and 'Donation Received Date'](#4.3)     
    4.3.1 [  Trend in 'Donations Amount'](#4.3.1)      
    4.3.2 [  Yearwise and Monthwise Total Donation Amount](#4.3.2)    



* 5  [Combining the 'Donor' and 'Donations' datasets](#5)    
    5.1 [Total Donations per state](#5.1)   
    5.2 [Total donations per city](#5.2)       
    5.3 [Count of donors from each state](#5.3)     
    5.4 [Donor Cart Sequence for 'Donor is Teacher'](#5.4)     



* 6  [Resources.csv](#6)    
    6.1 [A glimpse of Resources.csv](#6.1)    
    6.2 [Most requested items and vendors](#6.2)    
    6.2.1 [  Most popular items](#6.2.1)        
    6.2.2 [  Most popular vendors](#6.2.2)     
    6.3 [Most expensive items and items requested in bulk](#6.3)    
    6.3.1 [  Most expensive items](#6.3.1)    
    6.3.2 [  Items requested in bulk](#6.3.2)    
    6.4 [Vendors' earnings](#6.4)     
    6.4.1 [  Highest Grossing Vendors (total price for all units)](#6.4.1)    
    6.4.2 [  Highest Grossing Vendors per-unit price](#6.4.2)    



* 7 [Projects.csv](#7)   
    7.1 [A glimpse of Projects.csv](#7.1)    
    7.1.1 [  Percentage of projects of each project type](#7.1.1)    
    7.2 [Comparing 'Project Type' and 'Project Grade Level Category'](#7.2)    
    7.3 [Analyzing 'Teacher Project Posted Sequence'](#7.3)       
    7.3.1 [  Teacher Project Posted Sequence vs. Project Type](#7.3.1)    
    7.3.2 [  Teacher Project Posted Sequence vs. Project Grade Level Category](#7.3.2)    
    7.4 [Project Posted Date](#7.4)     
    7.4.1 [  Project Posted Date Trend](#7.4.1)         
    7.4.2 [  On what months are projects posted in different states](#7.4.2)      
    7.4.3 [  Comparing Total Project Cost and Project Posted Date](#7.4.3)               
    7.5 [Project Current Status information](#7.5)         
    7.6 [Average and Total Project Cost for different project categories](#7.6)        
    7.7 [Word Clouds](#7.7)         
    7.7.1 [  Word Cloud of Project Essay](#7.7.1)   
    7.7.2 [  Word Cloud of Project Title + Short Description](#7.7.2)    
    7.7.3 [  Comparing Fully Funded and Non Funded Project Title Word Clouds](#7.7.3)   
    7.7.4 [  Comparing Fully Funded and Non Funded Project Essay Word Clouds](#7.7.4)    
    7.7.5 [  Comparing Teacher-Led and Student-Led Project (Title + Description) Word Clouds](#7.7.5)    
    7.8 [Pre-Processing the Project Essay](#7.8)    
    7.8.1 [  Cleaning and lemmatization](#7.8.1)    
    7.8.2 [  Most common phrases used in Fully Funded and Non Funded Project Essays](#7.8.2)  
    7.9 ['Project Subject Category Tree' and 'Project Subject Subcategory Tree'](#7.9)



* 8  [Correlating  all datasets](#8)   
    8.1 [Mapping School States to Donor States](#8.1)    
    8.2 [Analyzing 'Project Cost' & 'Donation Amount'](#8.2)      
    8.2.1 [  wrt 'School Metro Type'](#8.2.1)       
    8.2.2 [  wrt 'Teacher Prefix'](#8.2.2)         
    8.3 [Average and Total Project Cost](#8.3)    
    8.3.1 [  wrt 'Project Type'](#8.3.1)    
    8.3.2 [  wrt 'Teacher Prefix'](#8.3.2)    
    8.4 [Trend of Teacher Prefix project posted year](#8.4)    
    8.5 [Obtaining the relation between the 'Project Cost' and the number of days taken to fully fund the project](#8.5)    
    8.6 [Relation between 'Project Resource Category', 'Donor State', 'School State', 'Donor City', 'School City'](#8.6)   
    8.7 [Fully-Funded to Expired ratio for each state](#8.7)   
    8.7.1 [  for different States](#8.7.1)    
    8.7.2 [  for 'School Metro Type'](#8.7.2)    
    8.7.3 [  for 'Donation Included Optional Donation'](#8.7.3)    
    8.7.4 [  for 'Teacher Prefix'](#8.7.4)    
    8.8 [Comparison of 'Donor Cart Sequence' with 'Donor State', 'Donor City' and 'School Name'](#8.8)   
    8.9 [Average 'Donor Cart Sequence' for different grades](#8.9)   
    8.10 [Analyzing 'Projects Resource Category'](#8.11)    
    8.10.1 [  Which states are the highest and lowest in terms of 'Donation Amount' for various 'Project Resource categories'](#8.11.1)    
    8.10.2 [  What 'Project Resource Category' do most states donate to:](#8.11.2)    
    8.11 ['Teacher Project Posted Sequence' for different states, cities and schools](#8.12)   
    8.13 ['Teacher Project Posted Sequence' and 'Donor Cart Sequence' for different Project Resource Categories](#8.13)     
    8.13 [Trend of Top 5 States in terms of Projects Posted per year](#8.14)    
    
    
    
* 9 [Summary](#9)     



* 10 [Recommendation System Model](#10)    
   10.1 [Popularity Based Model](#11.1)      
   10.2 [Content Based Model](#11.2)     
   10.3 [Collaborative Filtering](#10.1)         
   10.3.1 [Model Matrix](#10.2)    
   10.3.2 [Matrix completion using Singular Value Thresholding Algorithm](#10.2.1)    
   10.3.3 [Matrix Completion using Nonlinear Successive Over-Relaxation Algorithm](#10.2.2)   
   10.3.4 [ Feature Selection (Novel Idea)](#10.2.3)     
   10.4 [Multiple Content Feature Based Recommendation](#11.5)     
   10.5 [Hybrid Model](#11.6)
   
   

# 1. Teachers.csv 
<a id="1"></a>

## 1.1. A glimpse of Teachers.csv
<a id="1.1"></a>

In [2]:
# A sample of 'Teachers' data
teachers=pd.read_csv('../input/io/Teachers.csv')
teachers.describe()

* It is safe to replace 'null' values with 'Mrs.' since it is almost 50%. Also replace 'Mx.' with 'Ms.' assuming it to be a typo.





## 1.2 Teacher Prefix Pre-Processing
<a id="1.2"></a>

In [4]:
teachers['Teacher Prefix'].fillna('Mrs.', inplace=True)

# replacing 'Mx.' with 'Ms.' 
teachers['Teacher Prefix']=teachers['Teacher Prefix'].apply(lambda x: x.replace('Mx.','Ms.'))


count_prefix=Counter(teachers['Teacher Prefix'])
data = [Pie(labels=list(count_prefix.keys()),values=list(count_prefix.values()))]
#py.iplot(fig, filename='1')
print(count_prefix)
explode = ( 0.1, 0.05, 0.1, 0.05,0.1)
plt.figure(figsize=(8,8))
plt.rcParams['font.size'] = 12
plt.title('Teacher Prefix Information')
plt.pie(list(count_prefix.values()), explode=explode, labels=list(count_prefix.keys()), autopct='%1.1f%%',
        shadow=True)  

plt.show()

## 1.3 Trend of Teacher First Project Posted Date
<a id="1.3"></a>

In [5]:
# separating the date into day, month and year
def date(df):
    df = df.copy()
    df['Year']=pd.DatetimeIndex(df['Teacher First Project Posted Date']).year
    df['Month']=pd.DatetimeIndex(df['Teacher First Project Posted Date']).month
    df['Day']=pd.to_datetime(df['Teacher First Project Posted Date']).dt.weekday_name
    return df
teachers_modified= date(teachers)

In [13]:
teachers_modified['Day']=pd.Categorical(teachers_modified['Day'], categories=['Sunday','Monday','Tuesday' ,'Wednesday',
                                                                              'Thursday', 'Friday','Saturday'],ordered=True)
months={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep',10:'Oct', 11:'Nov',12:'Dec'}
teachers=teachers_modified.replace({'Month':months})

# sorting the months serially
teachers=Sort_Dataframeby_Month(teachers,monthcolumnname='Month')
count=pd.DataFrame.from_dict(Counter(teachers['Month']), orient='index').reset_index()
count=count.rename(columns={'index':'Month', 0:'count'})
sort=Sort_Dataframeby_Month(count, monthcolumnname='Month')

In [14]:
x1 = list(teachers_modified['Day'].value_counts().sort_index().keys())
y1 = list(teachers_modified['Day'].value_counts().sort_index())

x2 = list(Counter(teachers['Year']).keys()) 
z1 = list(Counter(teachers['Year']).values())

x=list(sort['Month'])
y=list(sort['count'])


trace1 = Scatter(x=x1, y=y1, fill='tozeroy', mode= 'lines+markers')
#data1= Bar(x=list(Counter(teachers['Year']).keys()), y=list(Counter(teachers['Year']).values()),marker=dict(color='#A2D5F2'),
           #width=0.25,name='Teacher First Project Posted Date (Yearly count)')
trace2 = Bar(x=x2, y=z1, marker=dict(color='#e993f9'))
trace3 = Scatter(x=x, y=y, fill='tozeroy', fillcolor = '#fcc45f', mode= 'none')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["<b>Daily Count</b>", "<b>Monthly Count</b>", "<b>Yearly Count</b>"])
fig.append_trace(trace1, 1, 1);
fig.append_trace(trace2, 1, 3);
fig.append_trace(trace3, 1, 2);


fig['layout'].update(height=350, showlegend=False, yaxis1=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis2=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ),yaxis3=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    )
                    );

py.iplot(fig, filename='3')

* **Teachers posted their first projects mostly on weekends** ** <span style='color:red'>(Saturday and Sunday)</span>.**

* **Most teachers posted their first project in the**

    **<span style='color:red'>Year :</span>** ** <span style='color:blue'>2016</span>   **        
    **<span style='color:red'>Month :</span>** **<span style='color:blue'>September</span>**

# 2. Schools.csv
<a id="2"></a>

## 2.1 A glimpse of Schools.csv
<a id="2.1"></a>

In [4]:
import numpy as np
schools = pd.read_csv('../input/io/Schools.csv', error_bad_lines=False)
schools.head(3)

### 2.1.1 Null values in the dataset
<a id="2.1.1"></a>

In [15]:
schools.apply(lambda x: sum(x.isnull()),axis=0)

### 2.1.2 Replacing the Null values in the 'School Percentage Free Lunch' with the School District average
<a id="2.1.2"></a>

In [28]:
#for schools without NCES data, a district average is used
grouped=schools.groupby('School District')
a=grouped['School Percentage Free Lunch'].agg(np.mean)
null_index= schools[schools['School Percentage Free Lunch'].isnull()].index.tolist()

for i in null_index:
   schools['School Percentage Free Lunch'][i]= a[schools['School District'][i]]
a[a.isnull()][:5]

* **For some districts, there are only null values available for 'School Percentage Free Lunch'. Hence the 'School Percentage Free Lunch' cannot be obtained.**

## 2.2 School Type Distribution
<a id="2.2"></a>

In [16]:
# @credit:Sban
words = " ".join(schools['School Name']).split()
from collections import Counter

school_types = Counter(words).most_common(30)[1:9]
school_types = [list(x) for x in school_types]
labels = [x[0]+" School" for x in school_types]
values = [x[1] for x in school_types]

fig=plt.figure(figsize=(20,5))
plt.rcParams['font.size'] = 15
plt.title('School Types Distribution')
plt.bar(labels,values, color='#ff00bb',width=0.25)
plt.ylabel('Count')
plt.show()

* **Most schools are** **<span style='color:red'>Elementary Schools</span>. **

# 3. Donors.csv
<a id="3"></a>

> ## 3.1 A glimpse of Donors.csv
<a id="3.1"></a>

In [3]:
donors = pd.read_csv('../input/io/Donors.csv')
donors.describe()

* **<span style='color:red'>Most donors are not teachers: 1910355</span>**

## 3.2 Using 'Donor Zip' to fill some 'nan' values in 'Donor State' 
<a id="3.2"></a>

In [20]:
#index of null values in 'Donor City' and 'Donor Zip'
#c=donors[donors['Donor City'].isnull()].index.tolist()
d=donors[donors['Donor Zip'].isnull()].index.tolist()

donors.iloc[134,:].to_frame(name=None).style.highlight_null(null_color='red')

** Seems like some samples have a (Donor Zip) without a (Donor City) like sample(134) **

### 3.2.1 replacing the 'Donor State' (other) for 'Donor Zip' (245) with 'Virginia'
<a id="3.2"></a>

In [4]:
ind= donors.loc[(donors['Donor State']=='other') & (donors['Donor Zip']=='245')].index.tolist()
for i in ind:
    donors['Donor State'][i]='Virginia'

# 4. Donations.csv
<a id="4"></a>

## 4.1 A glimpse of Donations.csv
<a id="4.1"></a>

In [5]:
donations= pd.read_csv('../input/io/Donations.csv')
donations.head(3)

## 4.2 Optional Donation information
<a id="4.2"></a>

In [23]:
# optional donation frequency
from collections import Counter
count_opt_donations= Counter(donations['Donation Included Optional Donation'])
explode = ( 0.1, 0.05)
plt.figure(figsize=(5,5))
plt.rcParams['font.size'] = 15
plt.title('Optional donation information')
plt.pie(list(count_opt_donations.values()), explode=explode, labels=list(count_opt_donations.keys()), autopct='%1.1f%%',
        shadow=True)  

plt.show()

** Most of the donations were included as** **<span style='color:red'>optional donations</span>.**

## 4.3 Comparing 'Total Donation Amount' and 'Donation Received Date'
<a id="4.3"></a>

In [24]:
# separating the date into day, month and year
def date(df):
    df = df.copy()
    df['Year']=pd.DatetimeIndex(df['Donation Received Date']).year
    df['Month']=pd.DatetimeIndex(df['Donation Received Date']).month
    df['Day']=pd.to_datetime(df['Donation Received Date']).dt.weekday_name
    return df

donations_modified= date(donations)
months={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep',10:'Oct', 11:'Nov',12:'Dec'}
donations=donations_modified.replace({'Month':months})

# sorting the months serially
donations=Sort_Dataframeby_Month(donations,monthcolumnname='Month')
count=pd.DataFrame.from_dict(Counter(donations['Month']), orient='index').reset_index()
count=count.rename(columns={'index':'Month', 0:'count'})
sort=Sort_Dataframeby_Month(count, monthcolumnname='Month')

### 4.3.1 Trend in 'Donations Amount '
<a id="4.3.1"></a>

In [25]:
donations['Day']=pd.Categorical(donations['Day'], categories=['Sunday','Monday','Tuesday' ,'Wednesday',
                                                                              'Thursday', 'Friday','Saturday'],ordered=True)
t4 = donations.groupby(['Day']).agg({'Year' : 'count', 'Donation Amount' : 'mean'}).reset_index().rename(columns={'Year' : 'Total Donations', 'Donation Amount' : 'Average Amount'}).sort_index()
x3 = t4['Day']
q1 = t4['Total Donations']
q2 = t4['Average Amount']

t1 = donations.groupby(['Year']).agg({'Month' : 'count', 'Donation Amount' : 'mean'}).reset_index().rename(columns={'Month' : 'Total Donations', 'Donation Amount' : 'Average Amount'})
x1 = t1['Year']
y1 = t1['Total Donations']
y2 = t1['Average Amount']

t2 = donations.groupby(['Month']).agg({'Year' : 'count', 'Donation Amount' : 'mean'}).reset_index().rename(columns={'Year' : 'Total Donations', 'Donation Amount' : 'Average Amount'})
# sorting the months serially
t2=Sort_Dataframeby_Month(t2, monthcolumnname='Month')
x2 = t2['Month']
z1 = t2['Total Donations']
z2 = t2['Average Amount']


trace1 = Scatter(x=x1[:-1], y=y1[:-1], fill='tozeroy', fillcolor = '#fcc45f', mode= 'none')
trace2 = Scatter(x=x1[:-1], y=y2[:-1], fill='tozeroy', fillcolor = "#e993f9", mode= 'none')
trace3 = Scatter(x=x2, y=z1, fill='tozeroy', fillcolor = 'lightseagreen', mode= 'none')
trace4 = Scatter(x=x2, y=z2, fill='tozeroy', fillcolor = "tomato", mode= 'none')
trace5 = Scatter(x=x3, y=q1, fill='tozeroy', fillcolor = 'coral', mode= 'none')
trace6 = Scatter(x=x3, y=q2, fill='tozeroy', fillcolor = "palevioletred", mode= 'none')

fig = tools.make_subplots(rows=3, cols=2, print_grid=False, subplot_titles = ["<b>Total Donation Amount per Year</b>", "<b>Average Donation Amount per Year</b>", "<b>Total Donation Amount per Month</b>", "<b>Average Donation Amount per Month</b>","<b>Donations per Day</b>", "<b>Average Donation Amount per Day</b>"])
fig.append_trace(trace1, 1, 1);
fig.append_trace(trace2, 1, 2);
fig.append_trace(trace3, 2, 1);
fig.append_trace(trace4, 2, 2);
fig.append_trace(trace5, 3, 1);
fig.append_trace(trace6, 3, 2);


fig['layout'].update(height=900, showlegend=False, yaxis1=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis2=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ),yaxis3=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis4=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
),yaxis5=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis6=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False)                     
                    );

py.iplot(fig, filename='multiple-subplots3') 

* **The number of donations on DonorsChoose has increased since** **<span style='color:blue'>2012</span>.**
* **The number of donations on DonorsChoose has increased at a higher rate since** **<span style='color:blue'>2015</span>.**
* **The average donations have gone down since** **<span style='color:blue'>2012</span> .**
* **Donations per Month drops drastically in the months of** **<span style='color:blue'>April, May, June & July</span>.**
* **The highest donations are recorded in the months of** **<span style='color:blue'>August & September</span>.**

### 4.3.2 Yearwise and Monthwise Total Donation Amount
<a id="4.3.2"></a>

In [29]:
# yearwise and monthwise total donation amount

z=donations[['Year','Month','Donation Amount']].groupby(['Year','Month'],as_index=False)['Donation Amount'].agg(np.sum)
z['Donation Amount']=z['Donation Amount'].apply(lambda x : int(x))
z =z.sort_values('Donation Amount', ascending=False)
st = 0

sizes = list(reversed([i for i in range(10,31)]))
intervals = int(len(z) / len(sizes))
size_array = [9]*len(z)

for i, size in enumerate(sizes):
    for j in range(st, st+intervals):
        size_array[j] = size 
    st = st+intervals
z['size_n'] = size_array
cols = list(z['size_n'])

# sorting the months serially
z=Sort_Dataframeby_Month(z, monthcolumnname='Month')

trace1 =Scatter( x=z['Month'], y=z['Year'], mode='markers', text=z['Donation Amount'],
        marker=dict( size=z.size_n, color=cols, colorscale='Picnic' ))
data = [trace1]
layout=dict(title='Total Donation Amount vs Donation Received Date')
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_pie1')

* The highest **Total Donation Amount** monthwise are:      
     **<span style='color:red'>January  :</span> **  ** <span style='color:blue'>2018</span>**            
     **<span style='color:red'>February :</span>**  ** <span style='color:blue'>2017</span>**            
     **<span style='color:red'>March    :</span> **  **<span style='color:blue'>2017</span>**         
     **<span style='color:red'>April    :</span>**  ** <span style='color:blue'>2018 </span> **        
     **<span style='color:red'>May      :</span>**  ** <span style='color:blue'>2017</span> **        
     **<span style='color:red'>June     :</span> **  **<span style='color:blue'>2017 </span>**          
     **<span style='color:red'>July     :</span> **  **<span style='color:blue'>2017</span>   **      
     **<span style='color:red'>August   :</span> **  **<span style='color:blue'>2016</span>  **        
     **<span style='color:red'>September:</span>**  ** <span style='color:blue'>2016</span>    **     
     **<span style='color:red'>October  :</span>**  ** <span style='color:blue'>2017 </span>     **     
     **<span style='color:red'>November :</span> **  **<span style='color:blue'>2017</span>   **        
     **<span style='color:red'>December :</span> **  **<span style='color:blue'>2017</span>  **    
    
        
* As is evident from the size of the blobs, hardly any donations are received in the months of  **<span style='color:red'>May, June & July</span>.  **

# 5. Combining the 'Donor' and 'Donations' datasets
<a id="5"></a>

<a id="5.1"></a>
## 5.1 Total Donations per state (in $)


In [31]:
z=pd.merge(donors[['Donor State','Donor ID']],donations[['Donation Amount','Donor ID']],on='Donor ID')
#donor_donations= pd.merge(donors,donations,on='Donor ID')
# state_wise average donations
grouped = z.groupby('Donor State', as_index=False)
state_wise_donations=grouped['Donation Amount'].agg(np.mean)
pd.set_option('float_format', '{:f}'.format)

# state wise total donations
# arranging the state_donations in descending order
top_donation_states=grouped['Donation Amount'].agg(np.sum).sort_values(
        ['Donation Amount'], ascending=False)

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]

top_donation_states['text'] = top_donation_states['Donor State'].astype(str) + '<br>' +\
    'Total donation amount:  $' + top_donation_states['Donation Amount'].astype(str)

    
state_to_code = {'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI', 'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 'Nevada': 'NV', 'Maine': 'ME'}
top_donation_states['State Code'] = top_donation_states['Donor State'].map(state_to_code)

data = [ dict(
        type = 'choropleth',
        locations = top_donation_states['State Code'],
        z = top_donation_states['Donation Amount'].astype(float),
        text = top_donation_states['text'],
        locationmode = 'USA-states',
        autocolorscale = False,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
        title = 'Donation amount in USD')) ]

layout = dict(
        title = 'State wise donations <br>(Hover for details) ',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-usa-map' )


<a id="5.2"></a>
## 5.2 Top states and cities in terms of Total Donation Amount(in $)


In [32]:
# city_wise average donation amount
z=pd.merge(donors[['Donor City','Donor ID']],donations[['Donation Amount','Donor ID']],on='Donor ID')
grouped= z.groupby('Donor City', as_index=False)
city_wise_donations=grouped['Donation Amount'].agg(np.mean)

# top cities in terms of total donations
top_donation_cities= grouped['Donation Amount'].agg(np.sum).sort_values(
        ['Donation Amount'], ascending=False)[:20]

In [33]:
trace1 = Bar(x=list(top_donation_cities['Donor City'][:20]), y=list(top_donation_cities['Donation Amount'][:20]),
              marker=dict(color='rgb(49,130,189)'), width=0.25, name='Top States in terms of donations')

trace2 = Bar(x=list(top_donation_states['Donor State'][:20]), y=list(top_donation_states['Donation Amount'][:20]),
              marker=dict(color='rgb(180,180,180)'), xaxis='x2',yaxis='y2', width=0.25, name='Top Cities in terms of donations')


data=[trace1, trace2]
layout = Layout(
    xaxis=dict(
        domain=[0, 1]
    ),
    yaxis=dict(
        domain=[0, 0.5]
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
    ),
    yaxis2=dict(
        domain=[0.55, 1],
        anchor='x2'
    ),
    
    height=600,
)

fig=dict(data=data, layout=layout)
py.iplot(fig, filename='6')

**Highest donations come from**     
*  **<span style='color:red'>State</span> ** **:**  ** <span style='color:blue'>California</span> ** ;                                  
*  **<span style='color:red'>City</span>  ** **:**   **<span style='color:blue'>New York</span> **

## 5.3 Count of donors from each state
<a id="3.2"></a>

In [34]:
count_donor= OrderedDict(Counter(donors['Donor State']).most_common(20))
plt.figure(figsize=(10,10))

plt.xticks(rotation=-90)
plt.rcParams['font.size'] = 10
plt.title('Total donors from each state',fontweight="bold")

plt.barh(list(count_donor.keys()),list(count_donor.values()), color='crimson')
plt.xlabel('Count')
plt.show()

In [35]:
count_donor_city= Counter(donors['Donor City'])
print('Cities with highest Donor Count:\n',count_donor_city.most_common()[1:10])

**Max donors from **     
 **<span style='color:red'>Donor City :</span> **   **<span style='color:blue'>(Chicago)=34352</span> **       
 **<span style='color:red'>Donor State:</span> **   **<span style='color:blue'>(California)=294695</span> **

## 5.4 Donor Cart Sequence for 'Donor is Teacher'
<a id="5.4"></a>

In [36]:
z=pd.merge(donors[['Donor Is Teacher','Donor ID']],donations[['Donor Cart Sequence','Donor ID']],on='Donor ID')
grouped=z.groupby(['Donor Is Teacher'], as_index=False)
grouped['Donor Cart Sequence'].agg(np.mean).style.set_properties(**{'background-color': 'black',
                           'color': 'aqua','border-color': 'white'})


* **Donor Cart Sequence** is higher for  **<span style='color:blue'>Teachers</span> ** as the same teachers donate repeatedly.

In [37]:
del donations_modified, teachers_modified

# 6. Resources.csv
<a id="6"></a>

## 6.1 A glimpse of Resources.csv
<a id="6.1"></a>

In [6]:
resources=pd.read_csv('../input/io/Resources.csv')
resources.head(3)

## 6.2 Most requested items and vendors 
<a id="6.2"></a>

### 6.2.1 Most popular items 
### 6.2.2 Most popular vendors
<a id="6.2.1"></a>
<a id="6.2.2"></a>

In [40]:
z=resources['Resource Item Name'].value_counts()[:20]
z1 = pd.DataFrame()
z1['item'] = z.index 
z1['Count'] =z.values
z1['Items'] = list(reversed("""Genius Kit, Acer 11'6 Chromebook, Noise, Kids' Wobble Chair, Seat Storage Sack,Kids Stay & Play Ball, Apple Ipad, HP 11'6 Chromebook,Black write and wipe markers , Kids Stay N Play Ball, Soft Seats, Privacy Partition,Apple Ipad, Commercial Furniture, Apple Ipad, Apple Ipad, Noise, Apple Ipad, Noise, Trip""".split(",")))
z1.groupby('Items').agg({'Count' : 'sum'}).reset_index().sort_values('Count',ascending=False)
z1 = z1[z1['Items'] != " Noise"]

x=resources['Resource Vendor Name'].value_counts()[:20]
x1 = pd.DataFrame()
x1['Vendor'] = x.index 
x1['Count'] =x.values

fig=plt.figure(figsize=(20,8))
fig.patch.set_facecolor('lightgray')
plt.subplot(1,2,1)
plt.xticks(rotation=-90)
plt.rcParams['font.size'] = 15
plt.title('Most Requested Items')
plt.bar(list(z1['Items']),list(z1['Count']), color='dodgerblue',width=0.25)
plt.ylabel('Count')
plt.grid(True)

plt.subplot(1,2,2)
plt.xticks(rotation=-90)
plt.rcParams['font.size'] = 15
plt.title('Most Popular Vendors')
plt.bar(list(x1['Vendor']),list(x1['Count']), color='blueviolet',width=0.25)
plt.ylabel('Count')
plt.grid(True)
plt.show()

*  **<span style='color:red'>Amazon Business</span> ** **is the most popular vendor.**
*  **<span style='color:red'>Trip</span> ** **is the most requested item.**

## 6.3  Most expensive items and items requested in bulk
<a id="6.3"></a>

### 6.3.1 Most expensive items 
### 6.3.2 Items requested in bulk
<a id="6.3.1"></a>
<a id="6.3.2"></a>

In [41]:
z=resources.sort_values(['Resource Quantity'],ascending=False)[:20]
z1 = pd.DataFrame()
z1['item'] = z['Resource Item Name']
z1['Quantity'] =z['Resource Quantity']
z1['Items'] = list(reversed("""Pencils, Dividers, Padlockcombinations, Spring Composition Book, Britannica School, Paw Print, patio stone, Paw Print, Paper, Bit Classroom Kit, Turintin Fbs, Eclipse Glasses, Math Subscription, Neck Ribbon, Floor Cover, Notebook, Rubber Flooring, Chair Glide, Renewable Glide, Crossling Blocks""".split(",")))
z1.groupby('Items')['Quantity'].agg(np.max).reset_index()

x=resources.sort_values(['Resource Unit Price'],ascending=False)[:20]
x1 = pd.DataFrame()
x1['item'] = x['Resource Item Name']
x1['Cost'] =x['Resource Unit Price']
x1['Items'] = list(reversed("""Ipad Cart, Home Theatre + Projector, Aromatic epdm, ddr Classroom Edition, Freeze Tag, Playground + Playstation, Fencing Supplies & Labor, Contrabassoon, Daktronic GS6, Leveled Bookroom, 10 Alpha, Playground + Playstation, Wood Playground Equipment, Commerical Structure, Sound System, Playstation, Telescoping Gym Bleachers, Commerical Structure, New Playground, Hadicapped Playground""".split(",")))
x1.groupby('Items')['Cost'].agg(np.max).reset_index()
plt.figure(figsize=(22,8))

plt.subplot(1,2,1)
plt.xticks(rotation=-90)
plt.rcParams['font.size'] = 15
plt.title('Most Expensive Items (per unit price)')
plt.barh(list(x1['Items']),list(x1['Cost']), color='mediumaquamarine')

plt.subplot(1,2,2)
plt.xticks(rotation=-90)
plt.rcParams['font.size'] = 15
plt.title('Items Requested In Bulk')
plt.barh(list(z1['Items']),list(z1['Quantity']), color='darkorange')

plt.show()

*  **<span style='color:red'>Playground for Handicapped people</span> ** **is the most expensive among all items.**
* **Items like**  **<span style='color:red'>Crossling Blocks</span> ** **are the highest requested items in bulk.**

<a id="6.4"></a>
## 6.4 Vendor's earnings (in $)

### 6.4.1 Highest Grossing Vendors (total price for all units)
### 6.4.2 Highest Grossing Vendors per-unit price
<a id="6.4.1"></a>
<a id="6.4.2"></a>

In [43]:
resources['Total Cost'] = resources['Resource Quantity'] * resources['Resource Unit Price']
z=resources[['Resource Vendor Name','Total Cost']]
z=z.dropna().groupby(['Resource Vendor Name'], as_index=False).agg(np.sum).sort_values('Total Cost', ascending=False)[:20]

z1=resources.sort_values('Resource Unit Price', ascending=False)[['Resource Vendor Name','Resource Unit Price']]
z1=z1.dropna().groupby(['Resource Vendor Name'], as_index=False).agg(np.mean).sort_values('Resource Unit Price', ascending=False)[:20]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.rcParams['font.size'] = 15
plt.title('Highest Grossing Vendors')
plt.barh(list(z['Resource Vendor Name']),list(z['Total Cost']), color='yellow')

plt.subplot(1,2,2)
fig.patch.set_facecolor('lightgray')
plt.rcParams['font.size'] = 15
plt.title('Highest Grossing Vendors per unit price')
plt.barh(list(z1['Resource Vendor Name']),list(z1['Resource Unit Price']), color='darkgray')
plt.show()

*  **<span style='color:red'>Amazon Business</span> ** **is the highest grossing vendor.**
*  **<span style='color:red'>MarkerBot</span> ** **is the highest grossing vendor on a per unit price basis.**
*  **<span style='color:red'>Best Buy Education</span> ** **not only has a very high per unit price but is also the second highest grossing vendor.**

# 7. Projects.csv
<a id="7"></a>

## 7.1 A glimpse of Projects.csv
<a id="7.1"></a>

In [6]:
projects=pd.read_csv('../input/io/Projects.csv')
projects.head(3)

1. ### 7.1.1 Percentage of projects of each project type
<a id="7.1.1"></a>

In [46]:
plt.figure(figsize=(8,8))
colors= ['gold', 'lightskyblue', 'lightcoral']
plt.rcParams['font.size'] = 12
explode = ( 0.3, 0.3, 0.3)
plt.title('Percentage of projects of each project type')
plt.pie(list(Counter(projects['Project Type']).values()), explode=explode, labels=list(Counter(projects['Project Type']).keys()), autopct='%1.1f%%',
        shadow=True, colors=colors)  

plt.show()

## 7.2 Comparing 'Project Type' and 'Project Grade Level Category'
<a id="7.2"></a>

In [47]:
grouped=projects[['Project Grade Level Category','Project Type']].groupby(['Project Type'], as_index=False)
grouped['Project Grade Level Category'].agg(lambda x: x.value_counts().index[0]).style.set_properties(**{'background-color': 'black',
                           'color': 'gold','border-color': 'white'})

* **In general,**  ** <span style='color:red'>Teacher-Led</span> **  **projects are for lower grades and**  **<span style='color:red'>Student-Led</span> ** ** Projects are mostly for higher grades.** 

In [48]:
grouped=projects[['Project Type','Project Grade Level Category']].groupby(['Project Grade Level Category'], as_index=False)
grouped['Project Type'].agg(lambda x: x.value_counts().index[0]).style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen','border-color': 'white'})

*  **<span style="color:blue"> In general, most projects are led by Teachers for all grades </span>. **

## Analyzing 'Teacher Project Posted Sequence'
<a id="7.3"></a>

### 7.3.1 Teacher Project Posted Sequence vs. Project Type
### 7.3.2 Teacher Project Posted Sequence vs. Project Grade Level Category
<a id="7.3.1"></a>
<a id="7.3.2"></a>

In [49]:
grouped=projects[['Teacher Project Posted Sequence','Project Type']].groupby('Project Type',as_index=False)
z=grouped['Teacher Project Posted Sequence'].agg(np.mean)
grouped1=projects[['Project Grade Level Category','Teacher Project Posted Sequence']].groupby(['Project Grade Level Category'], as_index=False)
z1=grouped1['Teacher Project Posted Sequence'].agg(np.mean).sort_values('Teacher Project Posted Sequence',ascending=False)

fig=plt.figure(figsize=[18,6])
plt.subplot(1,2,1)
plt.pie(z['Teacher Project Posted Sequence'],labels=z['Project Type'],autopct='%.0f%%', shadow=True)
plt.axis('equal')
plt.title('Teacher Project Posted Sequence \n vs \n Project Type',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.subplot(1,2,2)
plt.pie(z1['Teacher Project Posted Sequence'],labels=z1['Project Grade Level Category'],autopct='%.0f%%', shadow=True)
plt.axis('equal')
plt.title('Teacher Project Posted Sequence \n vs \n Project Grade Level Category',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.show()

* **On an average,** **Teacher Project Posted Sequence is higher for**  **<span style="color:blue"> Student-Led</span> ** **project type.** </span>.
* **On an average, Teacher Project Posted Sequence is higher for **  **<span style='color:red'>lower grades.</span> **

## 7.4 Project Posted Date
<a id="7.4"></a>

### 7.4.1 Project Posted Date Trend
<a id="7.4.1"></a>

In [51]:
projects['Day']=pd.to_datetime(projects['Project Posted Date']).dt.weekday_name
projects['Year']=pd.DatetimeIndex(projects['Project Posted Date']).year
projects['Month']=pd.DatetimeIndex(projects['Project Posted Date']).month

In [52]:
months={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep',10:'Oct', 11:'Nov',12:'Dec'}
projects=projects.replace({'Month':months})

# sorting the months serially
projects=Sort_Dataframeby_Month(projects,monthcolumnname='Month')
count=pd.DataFrame.from_dict(Counter(projects['Month']), orient='index').reset_index()
count=count.rename(columns={'index':'Month', 0:'count'})
sort1=Sort_Dataframeby_Month(count, monthcolumnname='Month')

projects['Day']=pd.Categorical(projects['Day'], categories=['Sunday','Monday','Tuesday' ,'Wednesday',
                                                                              'Thursday', 'Friday','Saturday'],ordered=True)
x3 = list(projects['Day'].value_counts().sort_index().keys())
y3 = list(projects['Day'].value_counts().sort_index())

x2 = list(Counter(projects['Year']).keys())
z1 = list(Counter(projects['Year']).values())

x=list(sort1['Month'])
y=list(sort1['count'])


trace1 = Bar(x=x3, y=y3)
trace2 = Bar(x=x2, y=z1, marker=dict(color='#e993f9'))
trace3 = Scatter(x=x, y=y, fill='tozeroy', fillcolor = '#fcc45f', mode= 'none')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["<b>Daily Count</b>", "<b>Monthly Count</b>", "<b>Yearly Count</b>"])
fig.append_trace(trace1, 1, 1);
fig.append_trace(trace2, 1, 3);
fig.append_trace(trace3, 1, 2);


fig['layout'].update(height=350, showlegend=False, yaxis1=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ), yaxis2=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ),yaxis3=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    )
                    );

py.iplot(fig, filename='100')

*  **<span style='color:red'>Most projects were posted on weekends.</span> ** ** This follows the same pattern as the first project posted by teachers.**
* Most projects were posted in the **year**,  **<span style='color:red'>2017</span>. **
* Most projects were posted in month of  **<span style='color:red'>September</span> ** just like **Teachers First Posted Project.**
* Least number of projects were posted in the months of  **<span style='color:red'>May</span> **  and  **<span style='color:red'>June.</span> **

* ### 7.4.2 On what months do are projects posted in different states
<a id="7.4.2"></a>


In [ ]:
# comparing School State and Project Posted Date
merged=pd.merge(projects[['Month','School ID']], schools[['School ID','School State']], on='School ID', how='inner')

grouped=merged[['Month','School State']].groupby(['School State'], as_index= False)
z=grouped.agg(lambda x: x.value_counts().index[0])
grouped=z[['Month','School State']].groupby(['Month'])
aug=grouped['School State'].apply(lambda x: sorted(set(x)))['Aug']
nov=grouped['School State'].apply(lambda x: sorted(set(x)))['Nov']
apr=grouped['School State'].apply(lambda x: sorted(set(x)))['Apr']
mar=grouped['School State'].apply(lambda x: sorted(set(x)))['Mar']
#Oct=grouped['School State'].apply(lambda x: sorted(set(x)))['Oct']
Sep=grouped['School State'].apply(lambda x: sorted(set(x)))['Sep']

z={'States':[apr,mar,aug,Sep,nov], 'Months': ['Apr','Mar','Aug','Sep','Nov']}
z=pd.DataFrame(z)
z.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen','border-color': 'white'})



**Table showing 'States' and their predominant 'Project Posted Months' .**

* Most States post their projects in the months of  **<span style='color:red'>August</span> **  and  **<span style='color:red'>September</span>. **

In [434]:
del merged

### 7.4.3 Comparing Total Project Cost and Project Posted Date
<a id="7.4.3"></a>

In [56]:
#projects['Project Cost']= projects['Project Cost'].replace({'\$':'', ',':''},regex=True).astype(float)

# yearwise and monthwise project cost

z=projects[['Year','Month','Project Cost']].groupby(['Year','Month'],as_index=False)['Project Cost'].agg(np.sum)
z['Project Cost']=z['Project Cost'].apply(lambda x : int(x))
z =z.sort_values('Project Cost', ascending=False)
st = 0

sizes = list(reversed([i for i in range(10,31)]))
intervals = int(len(z) / len(sizes))
size_array = [9]*len(z)

for i, size in enumerate(sizes):
    for j in range(st, st+intervals):
        size_array[j] = size 
    st = st+intervals
z['size_n'] = size_array
cols = list(z['size_n'])

# sorting the months serially
z=Sort_Dataframeby_Month(z, monthcolumnname='Month')

trace1 =Scatter( x=z['Month'], y=z['Year'], mode='markers', text=z['Project Cost'],
        marker=dict( size=z.size_n, color=cols, colorscale='Rainbow' ))
data = [trace1]
layout=dict(title='Total Project Cost vs Project Posted Date')
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='9')

* The highest **Total Project Cost** monthwise are:      
    **<span style='color:red'>January  :</span>** ** <span style='color:blue'>2018</span> **      
    **<span style='color:red'>February :</span>** ** <span style='color:blue'>2017</span> **    
    **<span style='color:red'>March    :</span>** ** <span style='color:blue'>2018</span> **      
    **<span style='color:red'>April    :</span>** ** <span style='color:blue'>2018 </span>     **   
    **<span style='color:red'>May      :</span>** ** <span style='color:blue'>2017</span>     **  
    **<span style='color:red'>June     :</span>** ** <span style='color:blue'>2017 </span>     **  
    **<span style='color:red'>July     :</span>** ** <span style='color:blue'>2017 </span>    **  
    **<span style='color:red'>August   :</span> ** **<span style='color:blue'>2016</span>     **   
    **<span style='color:red'>September:</span> ** **<span style='color:blue'>2014</span>    **   
    **<span style='color:red'>October  :</span> ** **<span style='color:blue'>2017 </span>     **  
    **<span style='color:red'>November :</span> ** **<span style='color:blue'>2017</span>     **  
    **<span style='color:red'>December :</span> ** **<span style='color:blue'>2017</span>      **  

## 7.5  Project Current Status information
<a id="7.5"></a>

In [306]:
# project current status
# obtaining relation between 'Project Cost' and 'Project Current Status'
grouped=projects[['Project Cost','Project Current Status']].groupby(['Project Current Status'],as_index=False)
z=grouped['Project Cost'].agg(np.mean)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
labels='Fully Funded','Expired','Archived','Live'
plt.pie([241329, 821367, 98003, 47952] , labels=labels,autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Project Current Status',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.subplot(1,2,2)
plt.title('Mean Project Cost   vs   Project Status')
barlist=plt.bar(z['Project Current Status'],z['Project Cost'], color='lightgray',width=0.15)
barlist[1].set_color('crimson')
plt.ylabel('Mean Project Cost')
plt.xlabel('Project Status')

plt.show()

* Projects with lower **Average Project Costs** generally end up being **<span style='color:red'>Fully Funded.</span>**
* **<span style='color:red'>Expired projects</span>**  are ones which have a **higher average project cost.**

## 7.6  Average and Total Project Cost for different project categories
<a id="7.6"></a>

In [59]:
grouped=projects[['Project Resource Category','Project Cost']].groupby(['Project Resource Category'], as_index=False)
z=grouped['Project Cost'].agg(np.mean).sort_values('Project Cost', ascending=False)
z=z.rename(columns={'Project Cost': 'Average Project Cost'})
z1=grouped['Project Cost'].agg(np.sum)
z1=z1.rename(columns={'Project Cost': 'Total Project Cost'})
z=pd.merge(z,z1,on='Project Resource Category')

fig, axes = plt.subplots(ncols=2, sharey=True)
fig.set_figheight(8)
fig.set_figwidth(15)
axes[0].barh(z['Project Resource Category'],z['Average Project Cost'], color='palegreen')
axes[0].set(title='Average Project Cost')

axes[1].barh(z['Project Resource Category'],z['Total Project Cost'], color='hotpink')
axes[1].set(title='Total Project Cost')

axes[0].invert_xaxis()
#fig.legend(loc='upper right ', frameon=False)

plt.show()

*  On an average, **<span style="color:red">Trips</span>** cost the highest.
*  If we look at the **total Project Cost** for each category, **<span style="color:red">Technology</span>** costs the highest.

## 7.7 Word Clouds 
<a id="7.7"></a>
### 7.7.1 Word Cloud of Project Essay
<a id="7.7.1"></a>

In [ ]:
text = projects['Project Essay'] 
# read the mask image
mask = np.array(Image.open('../input/cloudmasks/kid3.png'))

stopwords = set(STOPWORDS)
plt.figure(figsize=(20,15))

# generate word cloud
wc = WordCloud(background_color="white", max_words=1000, mask=mask,
               stopwords=stopwords,colormap='cool',max_font_size=90)
wc.generate(str(text))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title('Word cloud of Projects (Essay)',fontweight="bold")
plt.show()

### 7.7.2 Word Cloud of Project Title + Short Description
<a id="7.7.2"></a>

In [ ]:
text = projects['Project Title'] + ' ' +projects['Project Short Description'] 
# read the mask image
mask = np.array(Image.open('../input/cloudmasks/kid2.jpg'))

stopwords = set(STOPWORDS)
plt.figure(figsize=(20,15))

# generate word cloud
wc = WordCloud(background_color="white", max_words=1000, mask=mask,
               stopwords=stopwords,colormap='cool',max_font_size=90)
wc.generate(str(text))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title('Word cloud of Projects (Essay)',fontweight="bold")
plt.show()

### 7.7.3 Comparing Fully Funded and Non Funded Project Title Word Clouds
<a id="7.7.3"></a>

In [62]:
full_funded_projects = projects.loc[projects['Project Current Status'] == 'Fully Funded']
not_funded_projects  = projects[(projects['Project Current Status'] == 'Expired') ]

text = full_funded_projects['Project Title'] 
text1= not_funded_projects['Project Title'] 
# read the mask image
mask = np.array(Image.open('../input/cloudmasks/kid4.jpg'))

stopwords = set(STOPWORDS)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[20,15])

# generate word cloud
wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='magma',max_font_size=90)
wc.generate(str(text))
ax1.imshow(wc, interpolation='bilinear')
ax1.axis("off")
ax1.set_title('Word cloud of Fully Funded Projects (Title)',fontweight="bold")

wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='nipy_spectral',max_font_size=90)
wc.generate(str(text1))
ax2.imshow(wc, interpolation='bilinear')
ax2.axis("off")
ax2.set_title('Word Cloud of Non-Funded Projects (Title)',fontweight="bold")
fig.show()

* **Fully Funded** project Title has words like **<span style="color:blue">Books, Time, Kids, Life, Together, Click, Real, Little, Extra, Flipping, Construction.</span>**
* **Non Funded** project Title has words like ** <span style="color:blue">Projection, Leaders, Young, Focus, Future, Brand, Power, Doodling, Bring, New, Clear. </span>**

### 7.7.4 Comparing Fully Funded and Non Funded Project Essay Word Clouds
<a id="7.7.4"></a>

In [63]:
text = full_funded_projects['Project Essay'] 
text1= not_funded_projects['Project Essay'] 
# read the mask image
mask = np.array(Image.open('../input/cloudmasks/kid3.png'))

stopwords = set(STOPWORDS)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[20,15])

# generate word cloud
wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='magma',max_font_size=90)
wc.generate(str(text))
ax1.imshow(wc, interpolation='bilinear')
ax1.axis("off")
ax1.set_title('Word cloud of Fully Funded Projects (Essay)',fontweight="bold")

wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='nipy_spectral',max_font_size=90)
wc.generate(str(text1))
ax2.imshow(wc, interpolation='bilinear')
ax2.axis("off")
ax2.set_title('Word Cloud of Non-Funded Projects (Essay)',fontweight="bold")
fig.show()

* **Fully Funded** project Essay has words like **<span style="color:blue">World, want, need, peace, enough, remarkable, poorest, small, society, powerful, remember. </span>**
* **Non Funded** project Essay has words like **<span style="color:blue">country, eager, new, work, country, excited, energetic, stallion, wonderful, curiosity, welcome.</span>**

### 7.7.5 Comparing Teacher-Led and Student-Led Project (Title + Description) Word Clouds
<a id="7.7.5"></a>

In [64]:
teacher_led=projects[projects['Project Type']=='Teacher-Led']
student_led=projects[projects['Project Type']=='Student-Led']

text = teacher_led['Project Short Description']+ ' ' + teacher_led['Project Title']
text1= student_led['Project Short Description']+ ' ' + student_led['Project Title']
# read the mask image
mask = np.array(Image.open('../input/cloudmasks/kid4.jpg'))

stopwords = set(STOPWORDS)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[20,15])

# generate word cloud
wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='magma',max_font_size=90)
wc.generate(str(text))
ax1.imshow(wc, interpolation='bilinear')
ax1.axis("off")
ax1.set_title('Teacher Led Projects (Description+Title)',fontweight="bold")

wc = WordCloud(background_color="white", max_words=200, mask=mask,
               stopwords=stopwords,colormap='nipy_spectral',max_font_size=90)
wc.generate(str(text1))
ax2.imshow(wc, interpolation='bilinear')
ax2.axis("off")
ax2.set_title('Student Led Projects (Description+Title)',fontweight="bold")
fig.show()

* **Teacher-Led projects** have words like **<span style="color:blue"> diverse, middle, know, stud, often, excited, remember, need, technology, poverty, poorest, every, powerful. </span>**
* **Student-Led projects** have words like **<span style="color:blue">amazing, history, development, explore, benefit, risk, geometry, design, represent, inaugural. </span>**
* **Student-Led projects** have a lot of names in their word cloud (**<span style="color:red"> Kyra,Caroline,Karin,Isabella,Addam,Patricia etc </span>**).

## 7.8  Pre-Processing the Project Essay 
### 7.8.1 Cleaning and lemmatization
<a id="7.8"></a>
<a id="7.8.1"></a>

In [65]:
import string 
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

def clean_text(txt):
    # lower case
    txt = txt.lower()

    # punctuation removal 
    txt = ''.join(x for x in txt if x not in string.punctuation)

    # stopwords and lemmatization
    clean_txt = ""
    for word in txt.split():
        if word in stopwords:
            continue 
        clean_txt += " "
        clean_txt += word 
        
    noise = ['title','students','would', 'will','donotremoveessaydivider']
    for ns in noise:
        clean_txt = clean_txt.replace(ns, "")
        
    return clean_txt

year_18=projects[(projects['Year']==2018) | (projects['Month']=='March')]['Project Essay'].apply(clean_text)

### 7.8.2 Most common phrases used in Fully Funded and Non Funded Project Essays
<a id="7.8.2"></a>

In [69]:
def generate_ngrams(txt, N):
    grams = [txt[i:i+N] for i in range(len(txt)-N+1)]
    grams = [" ".join(b) for b in grams]
    return grams 

nf_18=projects[(projects['Project Current Status']=='Expired') & (projects['Year']==2018) | (projects['Month']=='March')]['Project Essay'].apply(clean_text)
ff_18=projects[(projects['Project Current Status']=='Fully Funded') & (projects['Year']==2018) | (projects['Month']=='March')]['Project Essay'].apply(clean_text)

nf_18['bigrams'] = nf_18.apply(lambda x : generate_ngrams(str(x).split(), 2))
ff_18['bigrams'] = ff_18.apply(lambda x : generate_ngrams(str(x).split(), 2))
nf_18['trigrams'] = nf_18.apply(lambda x : generate_ngrams(str(x).split(), 3))
ff_18['trigrams'] = ff_18.apply(lambda x : generate_ngrams(str(x).split(), 3))

all_bigrams_nf = []
for each in nf_18['bigrams']:
    all_bigrams_nf.extend(each)    
all_bigrams_ff = []
for each in ff_18['bigrams']:
    all_bigrams_ff.extend(each)

all_trigrams_nf = []
for each in nf_18['trigrams']:
    all_trigrams_nf.extend(each)    
all_trigrams_ff = []
for each in ff_18['trigrams']:
    all_trigrams_ff.extend(each)

del nf_18['bigrams'],ff_18['bigrams'],nf_18['trigrams'],ff_18['trigrams']  

In [70]:
z=Counter(all_bigrams_nf).most_common(20)
x_nf_bi=[a[0] for a in z]
y_nf_bi=[a[1] for a in z]

z=Counter(all_bigrams_ff).most_common(20)
x_ff_bi=[a[0] for a in z]
y_ff_bi=[a[1] for a in z]

z=Counter(all_trigrams_nf).most_common(20)
x_nf_tri=[a[0] for a in z]
y_nf_tri=[a[1] for a in z]

z=Counter(all_trigrams_ff).most_common(20)
x_ff_tri=[a[0] for a in z]
y_ff_tri=[a[1] for a in z]

fig=plt.figure(figsize=[25,20])
plt.rcParams.update({'font.size': 18})
fig.patch.set_facecolor('azure')

plt.subplot(2,2,1)
plt.barh(x_nf_bi,y_nf_bi,color='palevioletred')
b=plt.title('Most Common bigrams used for Non Funded Projects')

plt.subplot(2,2,2)
plt.barh(x_ff_bi, y_ff_bi,color='palevioletred')
a=plt.title('Most Common bigrams used for Funded Projects')

plt.subplot(2,2,3)
plt.barh(x_nf_tri,y_nf_tri,color='salmon')
b=plt.title('Most Common trigrams used for Non Funded Projects')

plt.subplot(2,2,4)
plt.barh(x_ff_tri, y_ff_tri,color='salmon')
a=plt.title('Most Common trigrams used for Funded Projects')

* **Apparently, Fully Funded projects largely highlight the financial challenges faced by students like**  **<span style='color:red'> low income, free food</span>**  **etc**.

## 7.9 'Project Subject Category Tree' and 'Project Subject Subcategory Tree'
<a id="7.9"></a>

In [282]:
# @ credit:sban
import operator
t = projects['Project Subject Category Tree'].value_counts()
x = list(t.index)
y = list(t.values)
r = {}
for i,val in enumerate(x):
    for each in val.split(","):
        x1 = each.strip()
        if x1 not in r:
            r[x1] = y[i]
        r[x1] += y[i]
sorted_x = list(sorted(r.items(), key=operator.itemgetter(1), reverse = True))[:10]
x1 = [a[0] for a in sorted_x][::-1]
y1 = [a[1] for a in sorted_x][::-1]



t1 = projects['Project Subject Subcategory Tree'].value_counts()
x2 = list(t1.index)
y2 = list(t1.values)
r = {}
for i,val in enumerate(x2):
    for each in val.split(","):
        x1_ = each.strip()
        if x1_ not in r:
            r[x1_] = y2[i]
        r[x1_] += y2[i]        
x2 = r.keys()
y2 = r.values()
sorted_x = list(sorted(r.items(), key=operator.itemgetter(1), reverse = True))[:10]
x2 = [a[0] for a in sorted_x][::-1]
y2 = [a[1] for a in sorted_x][::-1]

t = projects['Project Resource Category'].value_counts()
x3 = list(t.index[::-1])[:10]
y3 = list(t.values[::-1])[:10]

plt.figure(figsize=[20,8])
plt.subplot(1,3,1)
plt.title('Top Project Subject Category')
trace1 = plt.barh(x1, y1, color="#8cf2a9")
plt.subplot(1,3,2)
plt.title('Top Project Subject SubCategory')
trace2 = plt.barh(x2, y2,color="#8cf2a9")
plt.subplot(1,3,3)
trace3 = plt.barh(x3, y3, color="#8cf2a9")
plt.title('Top Project Resource Category')
plt.show()


In [ ]:
#del resources

# 8. Correlating  all datasets
<a id="8"></a>

## 8.1 Mapping School States to Donor States
<a id="8.1"></a>

In [348]:
# mapping school states to donor states
z=pd.merge(projects[['Project ID','School ID']],schools[['School ID','School State']],on='School ID')
z=pd.merge(z[['School State','Project ID']],donations[['Donor ID','Project ID','Donation ID','Donation Amount']],on='Project ID')
z=pd.merge(z[['School State','Donor ID','Donation ID','Donation Amount']],donors[['Donor ID','Donor State']],on='Donor ID')

pivot2 = z.pivot_table(columns='School State',
                                      index='Donor State', 
                                      values='Donation Amount', 
                                      aggfunc='sum',
                                      fill_value=0)

# Scale again by the funds that state receives
sum_state_funds = z.groupby('School State')['Donation Amount'].sum()
pivot2 = pivot2 / sum_state_funds.transpose()
pivot2=pivot2[pivot2.index != 'other']

In [349]:
#@credit:shep312
state_lat_lon = {
    'Alabama': [32.806671,-86.791130],
    'Alaska': [61.370716,-152.404419],
    'Arizona': [33.729759,-111.431221],
    'Arkansas': [34.969704,-92.373123],
    'California': [36.116203,-119.681564],
    'Colorado': [39.059811,-105.311104],
    'Connecticut': [41.597782,-72.755371],
    'Delaware': [39.318523,-75.507141],
    'District of Columbia': [38.897438,-77.026817],
    'Florida': [27.766279,-81.686783],
    'Georgia': [33.040619,-83.643074],
    'Hawaii': [21.094318,-157.498337],
    'Idaho': [44.240459,-114.478828],
    'Illinois': [40.349457,-88.986137],
    'Indiana': [39.849426,-86.258278],
    'Iowa': [42.011539,-93.210526],
    'Kansas': [38.526600,-96.726486],
    'Kentucky': [37.668140,-84.670067],
    'Louisiana': [31.169546,-91.867805],
    'Maine': [44.693947,-69.381927],
    'Maryland': [39.063946,-76.802101],
    'Massachusetts': [42.230171,-71.530106],
    'Michigan': [43.326618,-84.536095],
    'Minnesota': [45.694454,-93.900192],
    'Mississippi': [32.741646,-89.678696],
    'Missouri': [38.456085,-92.288368],
    'Montana': [46.921925,-110.454353],
    'Nebraska': [41.125370,-98.268082],
    'Nevada': [38.313515, -117.055374],
    'New Hampshire': [43.452492,-71.563896],
    'New Jersey': [40.298904,-74.521011],
    'New Mexico': [34.840515,-106.248482],
    'New York': [42.165726,-74.948051],
    'North Carolina': [35.630066,-79.806419],
    'North Dakota': [47.528912,-99.784012],
    'Ohio': [40.388783,-82.764915],
    'Oklahoma': [35.565342,-96.928917],
    'Oregon': [44.572021,-122.070938],
    'Pennsylvania': [40.590752,-77.209755],
    'Rhode Island': [41.680893,-71.511780],
    'South Carolina': [33.856892,-80.945007],
    'South Dakota': [44.299782,-99.438828],
    'Tennessee': [35.747845,-86.692345],
    'Texas': [31.054487,-97.563461],
    'Utah': [40.150032,-111.862434],
    'Vermont': [44.045876,-72.710686],
    'Virginia': [37.769337,-78.169968],
    'Washington': [47.400902,-121.490494],
    'West Virginia': [38.491226,-80.954453],
    'Wisconsin': [44.268543,-89.616508],
    'Wyoming': [42.755966,-107.302490]
}

flight_paths = []
for i in pivot2.index:
    
    for j in pivot2.columns:
        
        # Only plot if significant
        if (pivot2.loc[i, j] > 0.02) * (i != j):
               
            flight_paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',                           
                    lon = [state_lat_lon[i][1], state_lat_lon[j][1]],
                    lat = [state_lat_lon[i][0], state_lat_lon[j][0]],
                    mode = 'lines',
                    line = dict(
                        width = 10 * pivot2.loc[i, j],
                        color = 'red',                        
                    ),
                    text = '{:.2f}% of {}\'s donations come from {}'.format(100 * pivot2.loc[i, j], j, i),
                )
            )
    
layout = dict(
        title = 'Strongest out of state donation patterns (hover for details)',
        showlegend = False, 
        geo = dict(
            scope='usa',
            #projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            #subunitcolor = "rgb(217, 217, 217)",
            #countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
    
fig = dict(data=flight_paths, layout=layout)
py.iplot(fig,filename='d3-flight-paths')

* **Most donations come from the same state as the school state where the project was posted.**
* **Some School Projects from** ** <span style='color:blue'>New York, Texas, Illinois</span>**  etc. find Donors in **<span style='color:blue'>California</span>**, thus, making it the highest **Donating state in terms of Total Amount donated.**

## 8.2 Analyzing 'Project Cost' & 'Donation Amount'
<a id="8.2"></a>
### 8.2.1 wrt 'School Metro Type'  
<a id="8.2.1"></a>
### 8.2.2 wrt 'Teacher Prefix'  
<a id="8.2.2"></a>

In [371]:
merged=pd.merge(projects[['Project ID','Project Cost','School ID']], schools[['School ID','School Metro Type','School Percentage Free Lunch']],on='School ID',how='inner')
merged=pd.merge(merged[['Project Cost','School Metro Type','School Percentage Free Lunch','Project ID']], donations[['Donation Amount','Project ID']],on='Project ID', how='inner')

grouped=merged[['School Metro Type','Donation Amount']].groupby(['School Metro Type'], as_index=False)
z=grouped['Donation Amount'].agg(np.mean)

a=merged[merged['School Percentage Free Lunch']<20]['Donation Amount'].mean()
b=merged[merged['School Percentage Free Lunch'].between(20,40,inclusive=True)]['Donation Amount'].mean()
c=merged[merged['School Percentage Free Lunch'].between(40,60,inclusive=True)]['Donation Amount'].mean()
d=merged[merged['School Percentage Free Lunch'].between(60,80,inclusive=True)]['Donation Amount'].mean()
e=merged[merged['School Percentage Free Lunch']>80]['Donation Amount'].mean()

z1=[a,b,c,d,e]
z2=['below 20','20-40','40-60','60-80','80-100']

data1= Bar(x=list(z['School Metro Type']), y=list(z['Donation Amount']),marker=dict(color='#FDE724'), width=0.25)
data2 = Scatter(x = z2,y =z1,marker=dict(color='#35B778'))

grouped1=merged[['School Metro Type','Project Cost']].groupby(['School Metro Type'], as_index=False)
x1=grouped1['Project Cost'].agg(np.mean)

a=merged[merged['School Percentage Free Lunch']<20]['Project Cost'].mean()
b=merged[merged['School Percentage Free Lunch'].between(20,40,inclusive=True)]['Project Cost'].mean()
c=merged[merged['School Percentage Free Lunch'].between(40,60,inclusive=True)]['Project Cost'].mean()
d=merged[merged['School Percentage Free Lunch'].between(60,80,inclusive=True)]['Project Cost'].mean()
e=merged[merged['School Percentage Free Lunch']>80]['Project Cost'].mean()

z1=[a,b,c,d,e]

data3= Bar(x=list(x1['School Metro Type']), y=list(x1['Project Cost']),marker=dict(color='rgb(180,70,100)'), width=0.25)
data4= Scatter(x = z2,y =z1,marker=dict(color='rgb(100,70,120)'))

fig = tools.make_subplots(rows=2, cols=2, print_grid=False, subplot_titles = ['Donation Amount vs School Metro Type','Donation Amount vs School Percentage Free Lunch', 'Project Cost vs School Metro Type','Project Cost vs School Percentage Free Lunch'])
fig.append_trace(data1, 1, 1);
fig.append_trace(data2, 1, 2);
fig.append_trace(data3, 2, 1);
fig.append_trace(data4, 2, 2);

fig['layout'].update(height=700, showlegend=False, yaxis1=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ), yaxis2=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ),yaxis3=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ),yaxis4=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    )
                    );
py.iplot(fig, filename='11')

* On an average, **<span style="color:red">Urban</span>** and **<span style='color:red'>Suburban Projects</span>** receive the highest Donation Amount. 
* Schools with <span style="color:red">School Percentage Free lunch :</span> **<span style="color:blue">below 20%</span>** receive the highest average Donation Amount.
* **<span style="color:red">Urban and Suburban Projects</span>** have an average cost, higher than that of **<span style="color:red">Rural</span>** and ** <span style="color:red">Town.</span> **
* Schools with <span style="color:red">School Percentage Free lunch :</span> **<span style="color:blue">below 20%</span>** have a higher average Project Cost.
* The schools with **<span style="color:red">below 20%</span>**  **School Percentage Free Lunch** gets the highest **Donation Amount** and has a higher **Average Project Cost** because they mostly belong to **<span style='color:blue'>Urban</span>** and **<span style='color:blue'>Suburban</span>** areas which has the highest **Average Donation Amount** and **Average Project Cost.**
* The schools with **<span style="color:red">80-100%</span>**  **School Percentage Free Lunch** also  gets a higher **Donation Amount** and post a lower **Project Cost**.

In [ ]:
del merged

## 8.3 Average and Total Project Cost
<a id="8.3"></a>
<a id="8.3.1"></a>
<a id="8.3.2"></a>
### 8.3.1 wrt 'Project Type'
### 8.3.2 wrt 'Teacher Prefix'

In [374]:
grouped=projects[['Project Cost','Project Type']].groupby(['Project Type'], as_index=False)
z=grouped['Project Cost'].agg(np.sum)
z1=grouped['Project Cost'].agg(np.mean)

#comparing 'Project Cost' and 'Teacher Prefix'
x1=pd.merge(projects[['Project Cost','Teacher ID']],teachers[['Teacher ID','Teacher Prefix']],on='Teacher ID',how='inner')
grouped1=x1[['Project Cost','Teacher Prefix']].groupby(['Teacher Prefix'], as_index=False)
x1=grouped1['Project Cost'].agg(np.sum)
x2=grouped1['Project Cost'].agg(np.mean)

In [375]:
trace1 = Bar(
    x=x1['Teacher Prefix'],
    y=x1['Project Cost'], width=0.15,
    marker=dict(color='#22A784'),
   
)
trace2 = Scatter(
    x=x2['Teacher Prefix'],
    y=x2['Project Cost'],
    marker=dict(color='#79D151'),
    
)

trace3 = Bar(
    x=z['Project Type'],
    y=z['Project Cost'], width=0.15,
    
)
trace4 = Scatter(
    x=z1['Project Type'],
    y=z1['Project Cost'],
    
)

fig = tools.make_subplots(rows=2, cols=2, print_grid=False, subplot_titles = ['Total Project Cost vs. Teacher Prefix','Average Project Cost vs. Teacher Prefix', 'Total Project Cost vs. Project Type','Average Project Cost vs. Project Type'])
fig.append_trace(trace1, 1, 1);
fig.append_trace(trace2, 1, 2);
fig.append_trace(trace3, 2, 1);
fig.append_trace(trace4, 2, 2);

fig['layout'].update(height=700, showlegend=False, yaxis1=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ), yaxis2=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ),yaxis3=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    ),yaxis4=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=True
    )
                    );


py.iplot(fig, filename='multiple-subplots')

* Projects with **Teacher Prefix :**  **<span style='color:red'>Mrs. and Ms. (Female Teachers)</span>**  have highest **total Project Cost.**
* Projects with **Teacher Prefix :** ** <span style='color:red'>Mr. (Male Teachers)</span>**  have highest **average Project Cost.**
* **<span style='color:red'>Mrs. and Ms.</span>** carries the lowest **average Project Cost.**
* There are many **<span style='color:red'>Teacher-Led</span>** projects but the **average Project Cost** is higher for the **<span style='color:red'>Student-Led</span>** projects.

## 8.4 Trend of Teacher Prefix project posted year
<a id="8.4"></a>

In [388]:
# comparing the project posted year of 'Teacher Prefix' with the number of projects posted per year
merged=pd.merge(projects[['Year','Project ID','Teacher ID']],teachers[['Teacher Prefix','Teacher ID']],on='Teacher ID',how='inner')
merged=pd.merge(merged[['Year','Teacher Prefix','Project ID']], donations[['Donor ID','Project ID']],on='Project ID',how='inner')
grouped=merged[['Year','Teacher Prefix','Donor ID']].groupby(['Teacher Prefix','Year'], as_index=False)
z=grouped.agg({'Donor ID':'count'})
z=z[z['Year']!=2018]
data = []
data.insert(0,{'Year':2012, 'Donor ID':0})
cal=pd.concat([pd.DataFrame(data),z[z['Teacher Prefix']=='Mr.'][['Year','Donor ID']]],ignore_index=True)
tex=pd.concat([pd.DataFrame(data),z[z['Teacher Prefix']=='Mrs.'][['Year','Donor ID']]],ignore_index=True)
flo=pd.concat([pd.DataFrame(data),z[z['Teacher Prefix']=='Ms.'][['Year','Donor ID']]],ignore_index=True)
illi=pd.concat([pd.DataFrame(data),z[z['Teacher Prefix']=='Teacher'][['Year','Donor ID']]],ignore_index=True)
norCar=pd.concat([pd.DataFrame(data),z[z['Teacher Prefix']=='Dr.'][['Year','Donor ID']]],ignore_index=True)

fig = plt.figure(figsize=(20,8))
plt.rcParams['font.size'] = 15
plt.title('Teacher Prefix Project Posted Year: Trend')
ax1 = fig.add_subplot(111)

ax1.plot(cal['Year'], cal['Donor ID'],marker='o',label='Mr.',linewidth=2, markersize=12)
ax1.plot(tex['Year'], tex['Donor ID'],marker='o', label='Mrs.',linewidth=2, markersize=12)
ax1.plot(flo['Year'], flo['Donor ID'],marker='o', label='Ms.',linewidth=2, markersize=12)
ax1.plot(illi['Year'], illi['Donor ID'],marker='o', label='Teacher',linewidth=2, markersize=12)
ax1.plot(norCar['Year'], norCar['Donor ID'],marker='o', label='Dr.',linewidth=2, markersize=12)
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

* ** Number of Projects posted by**  **<span style='color:red'>female teachers</span>**  **grew by a huge extent as compared to their male counterparts.**

## 8.5 Obtaining the relation between the 'Project Cost' and the number of days taken to fully fund the project
<a id="8.5"></a>

In [389]:
# obtaining the relation between the 'Project Cost' and the number of days taken to fully fund the project
total_days=pd.to_datetime(projects['Project Fully Funded Date'])-pd.to_datetime(projects['Project Posted Date'])
x={'Project Cost':projects['Project Cost'], 'Total days': total_days}
x=pd.DataFrame(x)
grouped=x[['Project Cost','Total days']].groupby(['Total days'], as_index=False)
agg=grouped['Project Cost'].agg(np.mean)

In [390]:
ascend=agg.sort_values('Total days',ascending=False)[:50].mean()
descend=agg.sort_values('Total days',ascending=True)[:50].mean()
mean=agg.sort_values('Total days',ascending=True).mean()
z=agg.iloc[[1,10,30,50,70,100,130,150,170,180,200],:]

plt.figure(figsize=(20,8))
plt.rcParams['font.size'] = 15
plt.title('Project Cost   vs   Total days taken to fully fund the project')
plt.plot((z['Total days']/ np.timedelta64(1, 'D')).astype(int), z['Project Cost'], 'xb-')
#barlist[1].set_color('crimson')
plt.ylabel('Project Cost')
plt.xlabel('Total days taken to fund project')
plt.show()

* **On an average,**                
For project cost of **<span style='color:red'>951.8]</span>**,  it takes **<span style='color:blue'>41 days</span>** to fully fund it.     
For project cost of **<span style='color:red'>1584.37</span>**,  it takes** <span style='color:blue'>101 days</span>** to fully fund it.     
For project cost of **<span style='color:red'>2299.15</span>**,  it takes **<span style='color:blue'>182 days</span>** to fully fund it.     


* **As the project Cost increases, the number of days taken to fully fund the project also increases.**

## 8.6 Relation between 'Project Resource Category', 'Donor State', 'School State', 'Donor City', 'School City'
<a id="8.6"></a>

In [420]:
# relation between 'Project Resource Category', 'Donor State', 'School State', 'Donor City', 'School City'
merged=pd.merge(projects[['Project Resource Category','Project ID','School ID']], schools[['School City','School State','School ID']],on='School ID',how='inner')
merged=pd.merge(merged[['Project Resource Category','Project ID','School City','School State']],donations[['Project ID','Donor ID']],on='Project ID',how='inner')
merged=pd.merge(merged[['Project Resource Category','Donor ID','School City','School State']], donors[['Donor State','Donor City','Donor ID']], on='Donor ID',how='inner')

grouped=merged[['Project Resource Category','School State', 'Donor State', 'School City','Donor City']].groupby(['Project Resource Category'], as_index=False)
z=grouped.agg(lambda x: x.value_counts().index[0])



table_trace2 = Table(
    type='table',
    header = dict(height = 50,
                  values = ['<b>Project Resource Category</b>','<b>School State</b>','<b>Donor State</b>','<b>School City</b>' ,'<b>Donor City</b>'], 
                  line = dict(color='rgb(50, 50, 50)'),
                  align = ['center'],
                  font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                  fill = dict(color='#d562be')),
    cells = dict(values =[list(z.iloc[:,0]),list(z.iloc[:,1]),list(z.iloc[:,2]), list(z.iloc[:,3]), list(z.iloc[:,4])],
                 line = dict(color='#506784'),
                 align = 'center',
                 font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
                 height = 27,
                 fill = dict(color=['rgb(235, 193, 238)', 'rgba(228, 222, 249, 0.65)','rgba(228, 222, 249, 0.65)', 
                                    'rgba(118, 202, 219, 0.35)','rgba(118, 202, 219, 0.35)']))
)
layout=dict(title='Leading states for Project Resource Category <br>(Scroll for more information)')
fig=dict(data=[table_trace2], layout=layout)
py.iplot(fig, filename='13')

* **Schools** from **<span style='color:red'>California</span>** and **<span style='color:red'>New York states</span>** lead in **posting projects of all categories**.
* **<span style='color:red'>California</span>** leads in **project donations of all categories**.
* **<span style='color:blue'>Food, Clothing & Hygiene</span>** category is mostly from **schools** in **<span style='color:red'>New York</span>**  but the **donations** come mostly from **<span style='color:red'>California.</span>**
* **Schools** from **<span style='color:red'>New York City</span>** leads in **posting projects of all categories.**
* **<span style='color:blue'>Chicago city</span>** leads in **project donations of all categories.**

**Note: The above data is strictly on the basis of Count and not on the cost.**


## 8.7 Funded to Expired ratio
<a id="8.7"></a>
### 8.7.1  for different States
<a id="8.7.1"></a>

In [422]:
merged=pd.merge(projects[['Project Current Status','Project ID']],donations[['Project ID','Donor ID']],on='Project ID',how='inner')
merged=pd.merge(merged[['Project Current Status','Donor ID']], donors[['Donor State','Donor ID']], on='Donor ID',how='inner')

grouped= merged[['Project Current Status','Donor State']].groupby(['Donor State'], as_index=False)
z=grouped['Project Current Status'].agg(lambda x: sum(x=='Fully Funded'))
z1=grouped['Project Current Status'].agg(lambda x: sum(x=='Expired'))
z2=z['Project Current Status'].div(z1['Project Current Status'])
z3={'Donor State':z['Donor State'], 'Fully Funded':z['Project Current Status'] ,
                'Expired':z1['Project Current Status'], 'Funded to Expired ratio':z2}
z3=pd.DataFrame(data=z3)
z3=z3.sort_values('Funded to Expired ratio', ascending=False)


table_trace2 = Table(domain=dict(x=[0, 1],
                y=[0,0.4]),
    type='table',
    header = dict(height = 50,
                  values = ['<b>Donor State</b>','<b>Fully Funded</b>','<b>Expired</b>','<b>Funded to Expired ratio</b>'], 
                  line = dict(color='rgb(0, 0, 0)'),
                  align = ['center'],
                  font = dict(color=['rgb(45, 45, 45)'] * 4, size=14),
                  fill = dict(color='#FDE724')),
    cells = dict(values =[list(z3.iloc[:,0]),list(z3.iloc[:,2]),list(z3.iloc[:,1]), list(z3.iloc[:,3].apply(lambda x: round(x,3)))],
                 line = dict(color='rgb(0, 0, 0)'),
                 align = 'center',
                 font = dict(color=['rgb(0, 0, 0)'] * 4, size=14),
                 height = 27,
                 fill = dict(color=['#FCFEA4', '#CAB969','#FD9F6C', 
                                    '#a1dab4']))
)

trace1 = Bar(
    xaxis='x1',
    yaxis='y1',
    x=z3['Donor State'][:20],
    y=z3['Funded to Expired ratio'][:20],
    width=0.20,
    name='Top States',
    marker=dict(
        color='#FFE945'
    )
)

data = [table_trace2,trace1]
layout1 = dict(     
    #width=950,
    height=600,
    autosize=False,
    title='Funded to Expired projects of states (Scroll for more information)',
    margin = dict(t=100),
    showlegend=True,   
    #xaxis1=dict(domain=[0, 1],anchor='y1'),
    yaxis1=dict(domain=[0.6,1],anchor='x1'),
    plot_bgcolor='rgba(249,249, 249, 0.65)'
)


#layout=dict(title='<b>Comparison of Funded to Expired projects of each state<b>\n (Scroll for more information)')
fig=dict(data=data, layout=layout1)
py.iplot(fig, filename='14')

* **<span style='color:red'>District of Columbia</span>** has the best **Funded to Expired ratio**. **About 14 projects out of 15 are fully funded and one expires. But their number of projects are very less compared to the highest donating states.**
* **<span style='color:red'>Mississippi**</span>  has the worst ratio= **6.57**.
* **Among the top 5 highest donating states,** **<span style='color:red'>Illinois</span>** and **<span style='color:red'>New York</span>**  sport a good ratio.

In [ ]:
del z1,z2,z3

### 8.7.2 for 'School Metro Type'
<a id="8.7.2"></a>

In [411]:
merged=pd.merge(projects[['Project Current Status','School ID']], schools[['School ID','School Metro Type']], on='School ID',how='inner')

grouped= merged[['Project Current Status','School Metro Type']].groupby(['School Metro Type'], as_index=False)
z=grouped['Project Current Status'].agg(lambda x: sum(x=='Fully Funded'))
z1=grouped['Project Current Status'].agg(lambda x: sum(x=='Expired'))
z2=z['Project Current Status'].div(z1['Project Current Status'])
z3={'School Metro Type':z['School Metro Type'], 'Fully Funded':z['Project Current Status'] ,
                'Expired':z1['Project Current Status'], 'Funded to Expired ratio':z2}
z3=pd.DataFrame(data=z3)
z3=z3.sort_values('Funded to Expired ratio', ascending=False)

table_trace2 = Table(domain=dict(x=[0, 0.6],
                y=[0, 1.0]),
    type='table',
    header = dict(height = 30,
                  values = ['<b>School Metro Type</b>','<b>Funded to Expired ratio</b>'], 
                  line = dict(color='rgb(0, 0, 0)'),
                  align = ['center'],
                  font = dict(color=['#ffffff'] * 5, size=14),
                  fill = dict(color='#333333')),
    cells = dict(values =[list(z3['School Metro Type']), list(z3['Funded to Expired ratio'].apply(lambda x: round(x,4)))],
                 line = dict(color='rgb(0, 0, 0)'),
                 align = 'center',
                 font = dict(color=['rgb(0, 0, 0)'] * 5, size=12),
                 height = 27,
                 fill = dict(color=['#ffffff', '#cccccc']))
)


trace1 = Bar(
    xaxis='x1',
    yaxis='y1',
    x=z3['School Metro Type'],
    y=z3['Fully Funded'],
    name='Fully Funded projects', 
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace2 = Bar(
    xaxis='x1',
    yaxis='y1',
    x=z3['School Metro Type'],
    y=z3['Expired'],
    name='Expired projects',
    marker=dict(
        color='rgb(170,170,170)',
    )
)

axis=dict(
    showline=True,
    zeroline=False,
    showgrid=True,
    mirror=True,
    ticklen=4, 
    gridcolor='#ffffff',
    tickfont=dict(size=10)
)

fig=dict(data=[trace1 ,trace2], layout=layout)

data = [trace1, trace2,table_trace2]
layout1 = dict(
    width=950,
    height=500,
    autosize=False,
    margin = dict(t=100),
    showlegend=True,   
    xaxis1=dict(axis, **dict(domain=[0.65, 1], anchor='y1')),
    yaxis1=dict(axis, **dict(domain=[0.5, 1], anchor='x1', hoverformat='.2f')),  
    plot_bgcolor='rgba(228, 222, 249, 0.65)'
)

fig = Figure(data=data, layout=layout1)
py.iplot(fig, filename='2')

### 8.7.3 for 'Donation Included Optional Donation'
<a id="8.7.3"></a>
### 8.7.4 for 'Teacher Prefix'
<a id="8.7.4"></a>

In [ ]:
fig=plt.figure(figsize=[18,6])
plt.subplot(1,2,1)
merged=pd.merge(projects[['Project Current Status','Teacher ID']], teachers[['Teacher ID','Teacher Prefix']], on='Teacher ID',how='inner')

grouped= merged[['Project Current Status','Teacher Prefix']].groupby(['Teacher Prefix'], as_index=False)
z=grouped['Project Current Status'].agg(lambda x: sum(x=='Fully Funded'))
z1=grouped['Project Current Status'].agg(lambda x: sum(x=='Expired'))
z2=z['Project Current Status'].div(z1['Project Current Status'])
z3={'Teacher Prefix':z['Teacher Prefix'], 'Fully Funded':z['Project Current Status'] ,
                'Expired':z1['Project Current Status'], 'Funded to Expired ratio':z2}
z3=pd.DataFrame(data=z3)
z3=z3.sort_values('Funded to Expired ratio', ascending=False)
plt.pie(z3['Funded to Expired ratio'],labels=z3['Teacher Prefix'],autopct='%.0f%%', shadow=True)
plt.axis('equal')
plt.title('Project Current Status \n vs \n Teacher Prefix',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.subplot(1,2,2)
merged=pd.merge(projects[['Project Current Status','Project ID']], donations[['Project ID','Donation Included Optional Donation']], on='Project ID',how='inner')

grouped= merged[['Project Current Status','Donation Included Optional Donation']].groupby(['Donation Included Optional Donation'], as_index=False)
z=grouped['Project Current Status'].agg(lambda b: sum(b=='Fully Funded'))
z1=grouped['Project Current Status'].agg(lambda b: sum(b=='Expired'))
z2=z['Project Current Status'].div(z1['Project Current Status'])
z3={'Donation Included Optional Donation':z['Donation Included Optional Donation'], 'Fully Funded':z['Project Current Status'] ,
                'Expired':z1['Project Current Status'], 'Funded to Expired ratio':z2}
z3=pd.DataFrame(data=z3)
z3.sort_values('Funded to Expired ratio', ascending=False)
plt.pie(z3['Funded to Expired ratio'],labels=z3['Donation Included Optional Donation'],autopct='%.0f%%', shadow=True)
plt.axis('equal')
plt.title('Project Current Status \n vs \n Donation Included Option Donation',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()

* Schools in **<span style='color:red'>urban</span>**  areas have the highest **Funded to Expired ratio**. **<span style='color:red'>Towns</span>** have the lowest.
* Schools in **<span style='color:red'>urban</span>** areas also have more **Fully Funded projects**.   

* Prefix **<span style='color:red'>Dr.</span>** (Teacher Prefix) has the highest **Funded to Expired ratio**. 
* **Female teaches have a higher funded to expired ratio as compared to males.**
* **<span style='color:red'>Teacher</span>**  has the lowest.
* **<span style='color:red'>Not optional donation</span>** projects fetch a **higher Funded to Expired ratio** as compared to the projects which have **<span style='color:red'>optional donation</span>**.

## 8.8 comparison of 'Donor Cart Sequence' with 'Donor State', 'Donor City' and 'School Name'
<a id="8.8"></a>

In [412]:
#comparison of 'Donor Cart Sequence' with 'Donor State', 'Donor City' and 'School Name'.
merged=pd.merge(projects[['Project ID','School ID']], schools[['School Name','School ID']],on='School ID',how='inner')
merged=pd.merge(merged[['Project ID','School Name']],donations[['Project ID','Donor Cart Sequence','Donor ID']],on='Project ID',how='inner')
merged=pd.merge(merged[['Donor Cart Sequence','Donor ID','School Name']], donors[['Donor State','Donor City','Donor ID']], on='Donor ID',how='inner')

grouped=merged[['Donor Cart Sequence','Donor State','School Name','Donor City']].groupby(['Donor State','School Name','Donor City'], as_index=False)
z=grouped.agg(np.mean).sort_values('Donor Cart Sequence', ascending=False)[:30]
grouped1=merged[['Donor Cart Sequence','Donor State']].groupby(['Donor State'], as_index=False)
z1=grouped1.agg(np.mean).sort_values('Donor Cart Sequence', ascending=False)[:11]
z1=z1[z1['Donor State']!='other']
grouped2=merged[['Donor Cart Sequence','Donor City']].groupby(['Donor City'], as_index=False)
z2=grouped2.agg(np.mean).sort_values('Donor Cart Sequence', ascending=False)[:10]

table_trace2 = Table(domain=dict(x=[0, 1],
                y=[0, 0.7]),
    type='table',
    header = dict(height = 50,
                  values = ['<b>School Name<b>','<b>Donor City<b>','<b>Donor State</b>','<b>Mean Donor Cart Sequence</b>'], 
                  line = dict(color='rgb(0, 0, 0)'),
                  align = ['center'],
                  font = dict(color=['rgb(45, 45, 45)'] * 4, size=14),
                  fill = dict(color='#deebf7')),
    cells = dict(values =[list(z.iloc[:,1]),list(z.iloc[:,2]),list(z.iloc[:,0]),list(z.iloc[:,3].apply(lambda x: round(x,1)))],
                 line = dict(color='rgb(0, 0, 0)'),
                 align = 'center',
                 font = dict(color=['rgb(0, 0, 0)'] * 4, size=14),
                 height = 27,
                 fill = dict(color=['#4292c6', '#6baed6','#9ecae1', '#c6dbef']))
)


data = [table_trace2]
layout1 = dict(     
    #width=950,
    height=600,
    autosize=False,
    title='Mean Donor Cart Sequence \n (Scroll for more information)',
    margin = dict(t=100),
    showlegend=True,   
    plot_bgcolor='rgba(249,249, 249, 0.65)'
)


#layout=dict(title='<b>Comparison of Funded to Expired projects of each state<b>\n (Scroll for more information)')
fig=dict(data=data, layout=layout1)
py.iplot(fig, filename='1')

In [ ]:
plt.figure(figsize=[20,8])
plt.rcParams['font.size'] = 15
plt.subplot(1,2,1)
plt.title('Mean Donor Cart Sequence for top 10 states')
plt.barh(z1.iloc[:,0],z1.iloc[:,1].apply(lambda x: round(x,1)), color='#084594')
plt.subplot(1,2,2)
#plt.rcParams['font.size'] = 15
plt.title('Mean Donor Cart Sequence for top 10 cities')
plt.barh(z2.iloc[:,0],z2.iloc[:,1].apply(lambda x: round(x,1)), color='#a1dab4')
plt.show()

* **<span style='color:red'>Indiana</span>** has the highest number of donors who donate repeatedly, followed by **<span style='color:red'>Rhode Island</span>**.
* **<span style='color:red'>California</span>**, the highest donating state, comes **fifth**, with a **Mean Donor Cart Sequence** of about **one-third** of that of Indiana.
* **<span style='color:blue'>Manhattan Beach</span>** in **<span style='color:red'>California</span>**  enjoys the highest number of repeated donors, followed by **<span style='color:blue'>Zionsville</span>**  in **<span style='color:red'>Indiana</span>**  and **<span style='color:blue'>Coventry</span>** in **<span style='color:red'>Rhode Island.</span>**
* **Most schools in 'Manhattan Beach','California' enjoy high donor cart sequence.**

In [ ]:
del grouped1,grouped2

## 8.9 Average 'Donor Cart Sequence' for different grades
<a id="8.9"></a>

In [407]:
merged=pd.merge(projects[['Project Grade Level Category','Project ID']], donations[['Project ID','Donor Cart Sequence']], on='Project ID',how='inner')

grouped=merged[['Project Grade Level Category','Donor Cart Sequence']].groupby(['Project Grade Level Category'], as_index=False)
z=grouped['Donor Cart Sequence'].agg(np.mean).sort_values('Donor Cart Sequence',ascending=False)

fig=plt.figure(figsize=[6,6])
plt.axis('equal')
plt.pie(z['Donor Cart Sequence'] , labels=z['Project Grade Level Category'],autopct='%1.1f%%', shadow=True)
plt.axis('equal')
plt.title('Average Donor Cart Sequence',fontweight='bold')
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1.25)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.show()

* **<span style='color:red'>Lower grades</span>** have the higher **Donor Cart Sequence.**
* **<span style='color:red'>Grades 9-12</span>** have the lowest **Donor Cart Sequence.**

## 8.10 Analyzing 'Projects Resource Category'
<a id="8.11"></a>

### 8.10.1  Which states are the highest and lowest in terms of 'Donation Amount' for various 'Project Resource categories'
<a id="8.11.1"></a>

In [404]:
merged=pd.merge(projects[['Project Resource Category','Project ID']], donations[['Project ID','Donor ID','Donation Amount']], on='Project ID',how='inner')
merged=pd.merge(merged[['Project Resource Category','Donation Amount','Donor ID']], donors[['Donor State','Donor ID']], on='Donor ID',how='inner')

grouped= merged[['Donor State','Donation Amount','Project Resource Category']].groupby(['Project Resource Category','Donor State'], as_index=False)
df=grouped['Donation Amount'].agg(np.sum)
df_max = df.groupby('Project Resource Category').idxmax()
df_max['type'] = 'max'
df_min = df.groupby('Project Resource Category').idxmin()
df_min['type'] = 'min'

df2 = df_max.append(df_min).set_index('type',append=True).stack().rename('index')

df3 = pd.concat([ df2.reset_index().drop('Project Resource Category',axis=1).set_index('index'), 
                  df.loc[df2.values] ], axis=1 )

df3.set_index(['Project Resource Category','type']).sort_index().style.set_properties(**{'background-color': 'black',
                           'color': 'gold','border-color': 'white'})


* ** <span style='color:red'>California</span>**  leads in **Donation Amount** for **<span style='color:blue'>all</span>**  project categories except **<span style='color:blue'>Visitors</span>** category,  where **<span style='color:red'>New York</span>**  leads.
* **<span style='color:red'>Wyoming</span>**  and **<span style='color:red'>North Dakota</span>**  constitutes the lowest ranking states as far as **Donation Amount** is concerned.

In [405]:
del df_max,df_min,df2,df3

### 8.10.2 What 'Project Resource Category' do most states donate to:
<a id="8.11.2"></a>

In [403]:
merged=pd.merge(projects[['Project Resource Category','Project ID']], donations[['Project ID','Donor ID']], on='Project ID',how='inner')
merged=pd.merge(merged[['Project Resource Category','Donor ID']], donors[['Donor State','Donor ID']], on='Donor ID',how='inner')
grouped= merged[['Donor State','Project Resource Category']].groupby(['Donor State'], as_index=False)
z=grouped['Project Resource Category'].agg(lambda x: x.value_counts().index[0])
z=z.groupby(['Project Resource Category'])['Donor State'].apply(lambda x: sorted(set(x)))
z1=z[0]
z2=z[1]
pd.DataFrame({'Project Resource Category':['Supplies','Technology'], 'Donor States':[z1,z2]}).style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen','border-color': 'white'})

* **Most states donate to the project category** ** <span style="color:red">Supplies</span>.**

## 8.11  'Teacher Project Posted Sequence' for different states, cities and schools
<a id="8.12"></a>

In [418]:
# Comparing 'Teacher Project Posted Sequence'
merged=pd.merge(projects[['Teacher Project Posted Sequence','Project ID','School ID']], schools[['School Name','School ID']],on='School ID',how='inner')
merged=pd.merge(merged[['Teacher Project Posted Sequence','Project ID','School Name']],donations[['Project ID','Donor ID']],on='Project ID',how='inner')
merged=pd.merge(merged[['Teacher Project Posted Sequence','Donor ID','School Name']], donors[['Donor State','Donor City','Donor ID']], on='Donor ID',how='inner')

grouped=merged[['Teacher Project Posted Sequence','Donor State','School Name','Donor City']].groupby(['Donor State','School Name','Donor City'], as_index=False)
z=grouped.agg(np.mean).sort_values('Teacher Project Posted Sequence', ascending=False)[:30]
grouped1=merged[['Teacher Project Posted Sequence','Donor State']].groupby(['Donor State'], as_index=False)
z1=grouped1.agg(np.mean).sort_values('Teacher Project Posted Sequence', ascending=False)[:10]
grouped2=merged[['Teacher Project Posted Sequence','Donor City']].groupby(['Donor City'], as_index=False)
z2=grouped2.agg(np.mean).sort_values('Teacher Project Posted Sequence', ascending=False)[:10]

table_trace2 = Table(domain=dict(x=[0, 1],
                y=[0.55, 1]),
    type='table',
    header = dict(height = 50,
                  values = ['<b>School Name<b>','<b>Donor City<b>','<b>Donor State</b>','<b>Mean Teacher Project Posted Sequence</b>'], 
                  line = dict(color='rgb(0, 0, 0)'),
                  align = ['center'],
                  font = dict(color=['rgb(45, 45, 45)'] * 5, size=14),
                  fill = dict(color='#FBFCBF')),
    cells = dict(values =[list(z.iloc[:,1]),list(z.iloc[:,2]),list(z.iloc[:,0]),list(z.iloc[:,3].apply(lambda x: round(x,1)))],
                 line = dict(color='rgb(0, 0, 0)'),
                 align = 'center',
                 font = dict(color=['#ffffff'] * 5, size=12),
                 height = 27,
                 fill = dict(color=['#440154', '#30678D', '#35B778', '#30678D']))
)

trace1 = Bar(
    xaxis='x1',
    yaxis='y1',
    x=z1.iloc[:,0],
    y=z1.iloc[:,1].apply(lambda x: round(x,1)),
    name='Mean Teacher Project Posted Sequence for top 10 states', 
    width=0.25,
    marker=dict(
        color='#30678D'
    )
)

trace2 = Bar(
    xaxis='x2',
    yaxis='y2',
    x=z2.iloc[:,0],
    y=z2.iloc[:,1].apply(lambda x: round(x,1)),
    name='Mean Teacher Project Posted Sequence for top 10 cities', 
    width=0.25,
    marker=dict(
        color='#35B778'
    )
)

data = [table_trace2,trace1,trace2]
layout1 = dict(     
    #width=950,
    height=600,
    autosize=False,
    title='<b>Comparison of Mean Teacher Project Posted Sequence<b>\n (Scroll for more information)',
    margin = dict(t=100),
    showlegend=True,   
    xaxis1=dict(domain=[0, 0.5],anchor='y1'),
    yaxis1=dict(domain=[0,0.5],anchor='x1'),
    xaxis2=dict(domain=[0.55, 1],anchor='y2'),
    yaxis2=dict(domain=[0,0.5],anchor='x2'),
    plot_bgcolor='rgba(249,249, 249, 0.65)'
)


#layout=dict(title='<b>Comparison of Funded to Expired projects of each state<b>\n (Scroll for more information)')
fig=dict(data=data, layout=layout1)
py.iplot(fig, filename='4')

* **<span style="color:red">Idaho</span>** has the highest **Teacher Project Posted Sequence**.
* Among cities,  **<span style="color:red">Hadensville</span>** in  **<span style="color:blue">Virginia</span>**  has the highest **Teacher Project Posted Sequence**, followed by **<span style="color:red">Peach Bottom</span>** in **<span style="color:blue">Pennsylvania</span>**.
* **<span style="color:red">William McKinley Elementary School</span>** in **<span style="color:blue">Richmond, Virginia</span>** has the highest **Teacher Project Posted Sequence** among schools.
* **<span style="color:red">William McKinley Elementary</span>** schools in different cities lead in the **Teacher Project Posted Sequence** category.

## 8.12 'Teacher Project Posted Sequence' and 'Donor Cart Sequence' for different Project Resource Categories
<a id="8.13"></a>

In [402]:
grouped=projects[['Teacher Project Posted Sequence','Project Resource Category']].groupby('Project Resource Category',as_index=False)
z=grouped['Teacher Project Posted Sequence'].agg(np.mean)

merged=pd.merge(projects[['Project Resource Category','Project ID']], donations[['Project ID','Donor Cart Sequence']], on='Project ID',how='inner')
grouped=merged[['Donor Cart Sequence','Project Resource Category']].groupby('Project Resource Category',as_index=False)
z1=grouped['Donor Cart Sequence'].agg(np.mean)

#cm = sns.light_palette("green", as_cmap=True)
a=pd.merge(z,z1,on='Project Resource Category').sort_values('Teacher Project Posted Sequence',ascending=True)
#.style.background_gradient(cmap=cm)
plt.figure(figsize=[20,10])
plt.title('Comparison of Teacher Project Posted Sequence and Donor Cart Sequence with Project Resource Category')
plt.barh(a['Project Resource Category'],-a['Teacher Project Posted Sequence'],color='darkgrey',label='Teacher Project Posted Sequence')
plt.barh(a['Project Resource Category'],a['Donor Cart Sequence'],color='gold',label='Donor Cart Sequence')
plt.legend(loc='upper right ', frameon=False)
plt.show()

* **Teacher Project Posted Sequence** is highest for ** <span style="color:red">Food Clothing & Hygiene.</span>**
* **Donor Cart Sequence is highest** for ** <span style="color:red">Musical Instruments.</span>**
* **<span style="color:red">Flexible Seating & Technology</span>** is the lowest for both.

## 8.13 Trend of Top 5 States in terms of Projects Posted per year
<a id="8.14"></a>

In [395]:
# comparing the project posted year of states with the number of projects posted per year
merged=pd.merge(projects[['Year','Project ID']], donations[['Project ID','Donor ID']], on='Project ID',how='inner')
merged=pd.merge(merged[['Year','Donor ID']], donors[['Donor State','Donor ID']], on='Donor ID',how='inner')
grouped=merged[['Year','Donor State','Donor ID']].groupby(['Donor State','Year'], as_index=False)
z=grouped.agg({'Donor ID':'count'})
z=z[z['Year']!=2018]
data = []
data.insert(0,{'Year':2012, 'Donor ID':0})
cal=pd.concat([pd.DataFrame(data),z[z['Donor State']=='California'][['Year','Donor ID']]],ignore_index=True)
ny=pd.concat([pd.DataFrame(data),z[z['Donor State']=='New York'][['Year','Donor ID']]],ignore_index=True)
tex=pd.concat([pd.DataFrame(data),z[z['Donor State']=='Texas'][['Year','Donor ID']]],ignore_index=True)
flo=pd.concat([pd.DataFrame(data),z[z['Donor State']=='Florida'][['Year','Donor ID']]],ignore_index=True)
illi=pd.concat([pd.DataFrame(data),z[z['Donor State']=='Illinois'][['Year','Donor ID']]],ignore_index=True)

fig = plt.figure(figsize=(16,8))
plt.rcParams['font.size'] = 15
plt.title('Trend of top 10 Donor States')
ax1 = fig.add_subplot(111)

ax1.plot(cal['Year'], cal['Donor ID'],marker='o',label='California',linewidth=2, markersize=12)
ax1.plot(ny['Year'], ny['Donor ID'],marker='o', label='New York',linewidth=2, markersize=12)
ax1.plot(tex['Year'], tex['Donor ID'],marker='o', label='Texas',linewidth=2, markersize=12)
ax1.plot(flo['Year'], flo['Donor ID'],marker='o', label='Florida',linewidth=2, markersize=12)
ax1.plot(illi['Year'], illi['Donor ID'],marker='o', label='Illinois',linewidth=2, markersize=12)

plt.legend(loc='upper left')
plt.grid(True)
plt.show()

* **<span style='color:red'>California</span>** ** saw a large rise in the number of projects posted per year as compared to the other top 5 states.**

# 9. Summary
<a id="9"></a>

### School Information   
* Most schools are Elementary Schools.   
   
   
   


### Donor Information
* Most donations come from the same state as the school state where the project was posted.    
* We can observe that School Projects from New York, Texas, Illinois etc. find Donors in California, thus, making it the highest Donating state in terms of Total Amount donated.    
* California saw a large rise in the number of projects posted per year as compared to the other top 10 states.   
* Highest donations come from State: California ; City : New York     
* Max donors from Donor City : (Chicago)=34352; Donor State: (California)=294695    
* Most donors are not teachers= 1910355    
* Donor Cart Sequence is highest for Musical Instruments.       
* Flexible Seating is the lowest. has the lowest Donor Cart Sequence.   
* Lower grades have the higher Donor Cart Sequence.   
* Grades 9-12 have the lowest Donor Cart Sequence.    
* Indiana has the highest number of donors who donate repeatedly, followed by Rhode Island.      
* California, the highest donating state, comes fifth, with a Mean Donor Cart Sequence of about one-third of that of Indiana.   
* Manhattan Beach in California enjoys the highest number of repeated donors, followed by Zionsville in Indiana and Coventry in Rhode Island.   
* Most schools in Manhattan Beach','California' enjoy high donor cart sequence.   
* Donor Cart Sequence is higher for Teachers as the same teachers donate repeatedly.  






### Project Cost & Donation Amount      
* Urban and Suburban Projects have an average cost, higher than that of Rural and Town.     
* Schools with School Percentage Free lunch : below 20% have a higher average Project Cost.     
* The schools with below 20% School Percentage Free Lunch gets a higher Donation Amount and has a higher Average Project Cost because they mostly belong to Urban and Suburban areas which has the highest Average Donation Amount and Average Project Cost.     
* On an average,Urban and Suburban Projects receive the highest Donation Amount.     
* Schools with School Percentage Free lunch : below 20% receive the highest average Donation Amount.     






### Donation Information     
* The number of donations on DonorsChoose has increased since 2012.     
* The number of donations on DonorsChoose has increased at a higher rate since 2015.     
* The average donations have gone down since 2012 .     
* Donations per Month drops drastically in the months of April, May, June & July.     
* The highest donations are recorded in the months of August & September.     
* Most of the donations were included as optional donations.     






### Teacher Information     
* Number of Projects posted by female teachers has grown by a huge extent over the years as compared to their male counterparts.     
* Projects with Teacher Prefix : Mrs. have highest total Project Cost.     
* Projects with Teacher Prefix : Mr. have highest average Project Cost.     
* Mrs. carries the lowest average Project Cost.     
* There are many Teacher-Led projects but the average cost per Student-Led project is a lot higher.     
* Male teachers, on an average, post higher costs for projects.     
* Teachers posted their first projects mostly on weekends (Saturday and Sunday).     
* Most teachers posted their first project in the Year : 2016; Month : September     
* Teacher Project Posted Sequence is highest for Food Clothing & Hygiene.     
* Flexible Seating has the lowest Teacher Project Posted Sequence.    
* Idaho has the highest Teacher Project Posted Sequence.     
* Among cities, Hadensville in Virginia has the highest Teacher Project Posted Sequence, followed by Peach Bottom in Pennsylvania.   
* William McKinley Elementary School in Richmond, Virginia has the highest Teacher Project Posted Sequence among schools. 
* William McKinley Elementary schools in different cities lead in the Teacher Project Posted Sequence category.     
* On an average, Teacher Project Posted Sequence is higher for Student-Led project type.      
* On an average, Teacher Project Posted Sequence is higher for lower grades.     




    

### Project Information         
* Fully Funded project Title has words like Books, Time, Kids, Life, Together, Click, Real, Little, Extra, Flipping, Construction.      
* Non Funded project Title has words like Projection, Leaders, Young, Focus, Future, Brand, Power, Doodling, Bring, New, Clear.  
* Fully Funded project Essay has words like World, want, need, peace, enough, remarkable, poorest, small, society, powerful, remember.      
* Non Funded project Essay has words like country, eager, new, work, country, excited, energetic, stallion, wonderful, curiosity, welcome.      
* Teacher-Led projects have words like diverse, middle, know, stud, often, excited, remember, need, technology, poverty, poorest, every, powerful.      
* Student-Led projects have words like amazing, history, development, explore, benefit, risk, geometry, design, represent, inaugural.      
* Student-Led projects have a lot of names in their word cloud ( patricia, addam, anthony, annie, alicia ).      
* Teachers use phrases like free lunch, free breakfast, receive free etc.            
* Apparently, Fully Funded projects largely highlight the financial challenges faced by students like low income, free food etc.      
* In general, Teacher-Led projects are for lower grades and Student-Led Projects are mostly for higher grades.            
* In general, most projects are led by Teachers for all grades.            
* Most States post their projects in the months of August and September.      
* Most projects were posted on weekends. This follows the same pattern as the first project posted by teachers.      
* Most projects were posted in the year, 2017.      
* Most projects were posted in month of September just like Teachers First Posted Project.      
* Least number of projects were posted in the months of May and June.      
* Most states donate to the project category Supplies.      
* California leads in Donation Amount for all project categories except Visitors category, where New York leads.      
* Wyoming and North Dakota constitutes the lowest ranking states as far as Donation Amount is concerned.      
* Schools from California and New York states lead in posting projects of all categories.      
* California leads in project donations of all categories.      
* Food, Clothing & Hygiene category is mostly from schools in New York but the donations come mostly from California.      
* Schools from New York City leads in posting projects of all categories.      
* Chicago city leads in project donations of all categories.      
* On an average, Trips cost the highest.      
* If we look at the total Project Cost for each category, Technology costs the highest.      
* Prefix 'Dr.' (Teacher Prefix) has the highest Funded to Expired ratio. Female teaches have a higher funded to expired ratio as compared to males.
* 'Teacher' has the lowest Funded to Expired ratio.      
* Not optional donation projects fetch a higher Funded to Expired ratio as compared to the projects which have optional donation.      
* Schools in urban areas have the highest Funded to Expired ratio. Towns have the lowest.      
* Schools in urban areas also have more Fully Funded projects.      
* District of Columbia has the best Funded to Expired ratio. About 21 projects out of 22 are fully funded and one expires.
* Mississippi has the worst ratio= 6.57.      
* Among the top 5 highest donating states, Illinois and New York sport a good ratio of around 12.5.      
* For project cost of 951.8, it takes 41 days to fully fund it.      
* For project cost of 1584.37, it takes 101 days to fully fund it.      
* For project cost of 2299.15, it takes 182 days to fully fund it.      
* As the project Cost increases, the number of days taken to fully fund the project also increases.      
* Projects with lower Average Project Costs generally end up being Fully Funded.      
* Expired projects are ones which have a higher average project cost.     





### Resources Information      
* Amazon Business is the most popular vendor.      
* Trip is the most requested item.      
* Playgorund for Handicapped people is the most expensive among all items.      
* Items like Crossling Blocks are the highest requested items in bulk.      
* Amazon Business is the highest grossing vendor.      
* MarkerBot is the highest grossing vendor on a per unit price basis.      
* Best Buy Education not only has a very high per unit price but is also the second highest grossing vendor.      





# 10. Recommendation System Model
<a id="10"></a>

## 10.1 Simple Popularity based Model
<a id="11.1"></a>

In [ ]:
## Case 1: Recommend the most popular items
z=resources['Resource Item Name'].value_counts()[:20]
z1 = pd.DataFrame()
z1['item'] = z.index 
z1['Count'] =z.values
z1['Items'] = list(reversed("""Genius Kit, Acer 11'6 Chromebook, Noise, Kids' Wobble Chair, Seat Storage Sack,Kids Stay & Play Ball, Apple Ipad, HP 11'6 Chromebook,Black write and wipe markers , Kids Stay N Play Ball, Soft Seats, Privacy Partition,Apple Ipad, Commercial Furniture, Apple Ipad, Apple Ipad, Noise, Apple Ipad, Noise, Trip""".split(",")))
z1.groupby('Items').agg({'Count' : 'sum'}).reset_index().sort_values('Count',ascending=False)
z1 = z1[z1['Items'] != " Noise"]

z1[['Items','Count']]

In [ ]:
# Most popular vendors
x=resources['Resource Vendor Name'].value_counts()[:20]
x1 = pd.DataFrame()
x1['Vendor'] = x.index 
x1['Count'] =x.values
x1[['Vendor','Count']]

In [ ]:
del resources

## 10.2 Content-Based Recommender
<a id="11.2"></a>

In this section, you will try to build a system that recommends projects that are similar to a particular project. More specifically, we will compute pairwise similarity scores for all projects based on their project descriptions and recommend projects based on that similarity score.

The **Project Short Description** is available to you as the overview feature in our projects dataset. Let's inspect it:

In [87]:
# Short Description Based Recommender
projects['Project Short Description'].head()

In its current form, it is not possible to compute the similarity between any two descriptions. To do this, we need to compute the word vectors of each description or document.

We will compute **Term Frequency-Inverse Document Frequency (TF-IDF)** vectors for each document. This will give us a matrix where each column represents a word in the description vocabulary (all the words that appear in at least one document) and each column represents a project, as before.

In its essence, the TF-IDF score is the frequency of a word occurring in a document, down-weighted by the number of documents in which it occurs. This is done to reduce the importance of words that occur frequently in documents and therefore, their significance in computing the final similarity score.

In [88]:
# Term Frequency-Inverse Document Frequency (TF-IDF)
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
projects['Project Short Description'] = projects['Project Short Description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(projects['Project Short Description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

We see that **1110017 words** were used to describe **59057  projects**. With this matrix in hand, we can now compute a similarity score. There are several candidates for this; such as the Euclidean,  Jaccard and the cosine similarity scores. Again, there is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.


We will be using the **cosine similarity** to calculate a numeric quantity that denotes the similarity between two projects. We use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores).      

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score.     

We find the top projects similar to the project at index=0



In [90]:
tfidf_matrix[0]

In [91]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix for that particular index
def get_recommendation(index):
    cosine_sim = linear_kernel(tfidf_matrix[index], tfidf_matrix).flatten()
    return cosine_sim

In [92]:
#Construct a reverse map of indices and project titles
indices = pd.Series(projects.index, index=projects['Project ID']).drop_duplicates()

In [93]:
# related projects to the project at index=0
cosine_sim=get_recommendation(0)
projects.loc[list(cosine_sim.argsort()[:-5:-1])]

While this simple code does a decent job, it cannot capture the effect due to other features like Donor State, City, Project Type, Teacher Prefix etc. which has been found to affect the  donations in my analysis.

## 10.3 Collaborative Filtering
<a id="10.1"></a>

Recommender systems provide users with personalized suggestions for products or services. These systems often rely on Collaborating Filtering (CF), where past transactions are analyzed in order to establish connections between donors and projects. The three more successful approaches to CF 
* Latent factor models
* Neighborhood models
* Restricted Boltzmann Machines.

Neighborhood methods are centered on computing the relationships between projects or, alternatively, between donors. It can be done using a project oriented approach or a donor oriented approach. The first method transform donors to the project space. Neighborhood models are most effective at detecting very localized relationships. They rely on a few significant neighborhood relations, often failing to capture the vast majority of donations by donors.

Latent factor models, such as Singular Value Decomposition (SVD), comprise an alternative approach by transforming both projects and donors to the same latent factor space, thus making them directly comparable.  Latent factor models are generally effective at estimating overall structure that relates simultaneously to most or all projects. However, these models are generally effective at estimating overall structure that relates simultaneously to most or all projects.

A class of two-layer undirected graphical models, called Restricted Boltzmann Machines (RBM¡¯s), can be used to model tabular data. RBM¡¯s slightly outperform carefully-tuned
SVD models. When the predictions of multiple RBM models and multiple SVD models are linearly combined, it can give a better result. Learning is performed by using Contrastive Divergence method. **[Ruslan Salakhutdinov et al (Restricted Boltzmann Machines for Collaborative Filtering)]**

In order to combat overfitting due to the sparse matrix, regualrization of the parameters are done as proposed by **Yehuda Koren (Factorization Meets the Neighborhood: a Multifaceted Collaborative Filtering Model)**. This paper also combines both the neighbourhood and the latent factor models to achieve better results.

###  10.3.1 Model Matrix
<a id="10.2"></a>

A low dimensional feature vector can be assigned to both Projects as well as donors. Now, each donor will have a feature vector based on certain parameters. The same goes for projects so that the rating that each donor assigns to each movie is modeled by the scalar-product of the two feature vectors. This means that the N ¡Á M matrix  that N
donors assign to M projects is modeled by the matrix **X**. DonorsChoose.org would provide recommendation based on the following **(based on my analysis) :**

Whenever a donor donates for the first time, a new row can be added to the matrix containing values based on the above Donor Features. Similarly, when a project is posted, we get a  Project vector based on the Projects Features. This vector can be multiplied with matrix **X** to give a N  ¡Á 1 vector, each value of which specifies a single donor. The donors with maximum values can be recommeded that particular project.

Low-rank approximations based on minimizing the sum-squared distance can be found using Singular
Value Decomposition (SVD).  In the collaborative filtering domain, however, most of the data sets are sparse, giving rise to a difficult non-convex problem.

### 10.3.2 Matrix completion using Singular Value Thresholding Algorithm
<a id="10.2.1"></a>
There is a rapidly growing interest in the recovery of an unknown low-rank or approximately low-rank matrix from very limited information. consider the problem of recovering a data matrix from a sampling of its entries. This routinely comes up whenever one collects partially filled out surveys, and one would like to infer the many missing entries, as in our case of matching donors to projects. 

Singular Value Thresholding is applied takes minimum storage space and keeps the computation cost of each iteration low as given by **Jian-Feng Cai et al (A SINGULAR VALUE THRESHOLDING ALGORITHM FOR MATRIX COMPLETION).**  It basically does a nuclear norm minimization of the matix **X** by SVT using soft-thresholding operations on singular values of a certain matrix at each iteration.

### 10.3.3 Matrix Completion using Nonlinear Successive Over-Relaxation Algorithm
<a id="10.2.2"></a>
Nuclear norm minimization  which requires computing singular value decompositions, can be increasingly costly as matrix size and rank increases. To improve the capacity of solving large-scale problems, **Zaiwen Wen et al (SOLVING A LOW-RANK FACTORIZATION MODEL FOR MATRIX COMPLETION
BY A NONLINEAR SUCCESSIVE OVER-RELAXATION ALGORITHM)** proposed a low-rank factorization model with a nonlinear successive over-relaxation algorithm that requires only solving a linear least squares problem per iteration which has speed several times faster than nuclear-norm minimization because of avoiding SVD computatio all together.



### 10.3.4 Feature Selection (Novel Idea)
<a id="10.4.3"></a>
Feature selection can be applied before finding the matri **X** since it becomes computationally expensive to store and use many features (with the labelling of features, the vector size increases).  A **Game Theory** aproach with the help of **Shapley value**  for ranking the features can be implemented.  In this approach, each feature is treated as a player of a game and the idea is to evaluate the marginal contribution of every feature to the classification accuracy by using Shapley value and then eliminate
those features whose marginal contribution is less than a certain threshold. Since the calculation of the Shapley value involves summing over all possible permutations of the features, it becomes impractical to determine the Shapley value if the feature set size is large. It was therefore proposed to use multi perturbation approach of approximating the Shapley value **A Keinan  (Axiomatic scalable neurocontroller analysis via shapley value)**. Another approach could be **Nash Stability Partition (NSP)** as given in **Dinesh Garg (A Game Theoretic Approach for Feature Clustering and Its Application to Feature Selection)**.

In [7]:
# collaborative filtering based model
# finding the weighed score by Inverse Donor Frequency
grouped=donations[['Project ID','Donor ID','Donation Amount']]
#grouped['Donation Amount']=grouped['Donation Amount']/grouped['Donation Amount'].max()
grouped=grouped.groupby(['Project ID','Donor ID'],as_index=False)
# using donor frequency

score=grouped['Donation Amount'].agg(np.mean)

# normalized values
score['Donation Amount']=score['Donation Amount']/score['Donation Amount'].std()

In [9]:
score.head(5)

In [16]:
score[1:10].pivot(index='Project ID', columns='Donor ID', values='Donation Amount').fillna(0).as_matrix()

In [17]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


### SVD++ (Matrix Completion using Nonlinear Successive Over-Relaxation Algorithm 10.2.2)

In [30]:
## SVD++
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import evaluate

def SVDPP(data):
    # Use the famous SVD algorithm.
    algo = SVDpp()
    #data = Dataset.load_builtin(merged[['Project ID','Donor ID','score']][1:100])
    
    reader=Reader()
    data=Dataset.load_from_df(data, reader)    

    kf = KFold(n_splits=2)

    algo = SVDpp()
    evaluate(algo, data, measures=['RMSE', 'MAE'])
    # train and test algorithm.
    trainset = data.build_full_trainset()
    algo.train(testset)
    predictions = algo.test(trainset)

    top_n=get_top_n(predictions, n=2)


    for x,y in top_n.items():
        print(x,[z for (z,_) in y])



## 10.4 Multiple feature based recommendation
<a id="11.5"></a>

It goes without saying that the quality of our recommender would be increased with the usage of more features. That is exactly what we are going to do in this section. We are going to build a recommender based on the following data: ** Project Short Description, Project Title, Project Type, Donor City, Donor State, Teacher Prefix**'

These features are not available in our current dataset so the first step would be to load and merge them into your main DataFrame.

My data analysis shows that states where projects are posted fetch maximum donations from the very state.    
Hence we can consider either 'School State' or 'Donor State' as our feature

In [8]:
merged= pd.merge(projects[['Project Short Description','Project ID','Teacher ID','Project Title','Project Type']], donations[['Donor ID','Project ID']], on='Project ID')
merged=pd.merge(merged, donors[['Donor ID','Donor State','Donor City']], on='Donor ID')
merged=pd.merge(merged, teachers[['Teacher Prefix','Teacher ID']],on='Teacher ID')

We drop all rows with NAN values (All donors haven't donated to all projects).

In [104]:
# drop all rows with Nan values
merged=merged.dropna()
merged.head(5)

In [105]:
merged.shape

We are now in a position to create your feature soup", which is a string that contains all the features that we want to feed to our vectorizer.     

Taking a chunk of first 10000 rows.

In [107]:
# using multiple features
merged=merged[1:10000]
def create_soup(x):
    return ' '.join([x['Donor State']]) + ' ' + ' '.join([x['Teacher Prefix']])+ ' ' + ' '.join([x['Project Type']])+ ' ' + ' '.join([x['Project Short Description']]) + ' ' + ' '.join([x['Project Title']]) + ' ' + ' '.join([x['Donor City']])
merged['soup'] = merged.apply(create_soup, axis=1)

One important difference is that we use the **CountVectorizer** instead of TF-IDF. This is because we do not want to down-weight the presence of a city/state if the city/state features in lesser number of samples. It doesn't make much intuitive sense.

In [108]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(merged['soup'])

In [112]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendation(index):
    cosine_sim2 = cosine_similarity(count_matrix[index], count_matrix)
    return cosine_sim2

In [113]:
# Reset index of your main DataFrame and construct reverse mapping as before
merged = merged.reset_index()
indices = pd.Series(merged.index, index=merged['Project Title'])
#indices = pd.Series(merged.index, index=merged['Project ID']).drop_duplicates()

In [114]:
# related projects to the project at index=0
cosine_sim2=get_recommendation(0)
merged.loc[cosine_sim2.argsort()[0][:-5:-1]]

We see that our recommender has been successful in capturing more information due to more features and has given us (arguably) better recommendations. There are, of course, numerous ways of playing with this system in order to improve recommendations.    

* ** We can mention the Donor State, Donor City, Teacher Prefix, Project Type  muliple times to increase the weight of these features.**
* **Some more features like Project Resource Category, Resource Vendor Name, School Name etc.  can be included, which also affect the donations as is shown in the data analysis.**


## 10.5 Hybrid Model
<a id="11.6"></a>
In this section, I will try to build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

Input: Donor ID and the Project ID
Output: Similar projects sorted on the basis of expected score by that particular donor.

In [ ]:
from surprise import Reader, Dataset, SVD, evaluate

In [ ]:
def hybrid(userId, projectId):
    merged=pd.merge(merged,score[['Project ID','Donation Amount']],on='Project ID')
    #merged.rename(columns={'Donation Amount':'Score'})
    idx = indices[title]
        
    sim_scores = list(enumerate(cosine_sim2[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    project = merged.iloc[movie_indices]
    project['est'] = project['Project ID'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['Project ID']).est)
    project = project.sort_values('est', ascending=False)
    return project.head(10)